# pyQuAcqV7

## Install necessary tools

In [ ]:
#!pip install ortools
!pip install exact
#!pip install python-sat
!pip install cpmpy
!pip install gdown

In [ ]:
import cpmpy as cp

## Define operators

In [ ]:

import operator
# Here define constraints
# this is the relation: a+b!=c
def someAndDiff(a,b,c):
  return operator.ne( operator.add(a,b) , c )

def alldiff():
  pass

def allequ():
  pass

def eqDist1(x,y):

  return operator.eq(operator.abs(operator.sub(x,y)), 1)

def neDist1(x,y):

  return operator.ne(operator.abs(operator.sub(x,y)), 1)



def eqDist(a,b,c,d):
  return operator.eq( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

def neDist(a,b,c,d):
  return operator.ne( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))


def eqDist3(repeat,b,c):
  return operator.eq( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))

def neDist3(repeat,b,c):
  return operator.ne( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))



vals = [1,2,3,4,5]
ops = [operator.ge,operator.le,operator.lt,operator.gt,operator.ne,operator.eq]


def eqConst(x, const):
  return operator.eq(x,const)

def geConst(x, const):
  return operator.ge(x,const)

def leConst(x, const):
  return operator.le(x,const)

def ltConst(x, const):
  return operator.lt(x,const)

def gtConst(x, const):
  return operator.gt(x,const)

def neConst(x, const):
  return operator.ne(x,const)


def DistgtValue(x,y, const):
  return operator.gt(operator.abs(operator.sub(x,y)),const)

def DistgeValue(x,y, const):
  return operator.ge(operator.abs(operator.sub(x,y)),const)

def DistltValue(x,y, const):
  return operator.lt(operator.abs(operator.sub(x,y)),const)

def DistleValue(x,y, const):
  return operator.le(operator.abs(operator.sub(x,y)),const)

def DistneValue(x,y, const):
  return operator.ne(operator.abs(operator.sub(x,y)),const)

def DisteqValue(x,y, const):
  return operator.eq(operator.abs(operator.sub(x,y)),const)


def eqDist(a,b,c,d):
  return operator.eq( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

def neDist(a,b,c,d):
  return operator.ne( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

def eqDist3(repeat,b,c):
  return operator.eq( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))

def neDist3(repeat,b,c):
  return operator.ne( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))


def gtDist(a,b,c,d):
  return operator.gt( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

def geDist(a,b,c,d):
  return operator.ge( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

def gtDist3(repeat,b,c):
  return operator.gt( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))

def geDist3(repeat,b,c):
  return operator.ge( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))


def ltDist(a,b,c,d):
  return operator.lt( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

def leDist(a,b,c,d):
  return operator.le( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

def ltDist3(repeat,b,c):
  return operator.lt( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))

def leDist3(repeat,b,c):
  return operator.le( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))



## Projection

In [ ]:
def projection(e,vars_ids):
  # this is a projection of Vars on assignement: return the part of the assignement
  # including only Vars

  ids = [ee[0] for ee in e]
  els = { ee[0]: ee for ee in e}
  p = []

  for id in vars_ids:
    if id in ids:
      p.append(els[id])

  return p

## Constraint

In [ ]:
import operator
from copy import deepcopy
class Constraint:
    # this class represents a constraint

    def __init__(self ,scope_ids , rel, arity, isCommutative , parameters = None, isNegation=False, isRedundant=False):
      """
          scope_ids : list of variable ids (e.g. [1,2])
          rel : the relation of this constraint (e.g. operator.ne)
          arity : the arity of the constraint (e.g. 2)
          isCommutative : is this constraint commutative (e.g. True (A != B <=> B != A))
          parameters : list of parameters of this constraint (e.g. A>1 the parameters = [1])
          isNegation : whether this constraint is a negation of the constraint with the same parameters, in other words
          all parameters are the same except the relation which will be replaced by the opposit relation, this will be
          implemented with returning the not of verify later
      """
      assert isinstance(scope_ids,list), "The scope must be a list of variable ids"
      for x in scope_ids:
        assert isinstance(x,int), "The scope must be a list of variable ids"

      self.scope_ids = scope_ids

      self.isCommutative = isCommutative

      #The relation must be an operator
      assert callable(rel), "The relation must be an operator"
      self.rel = rel

      # the arity is the number of variables in the scope, an arity could be binary, ternary ...
      self.arity = arity

      self.parameters = parameters

      self.id = id(self)

      self.isNegation = isNegation
      self.isRedundant = isRedundant

    def verify(self, assignement):
      # this method checks whether the constraint accepts an assignement (True) or rejects it (False)

      # we project the assignement on the scope of the constraint
      p = projection(assignement, self.scope_ids)

      # get values of vars in the assignement
      #print("P: ", p)
      if(p==[]):
        return True # the constraint is not concerned with this assignement

      values = [pp[2] for pp in p]
      # print("values hereeee _________",values)
      # print("Self", self)
      # print("self.rel", self.rel)
      # print("self.scope_ids", self.scope_ids)

      if(len(values) != self.arity and self.arity != None):
        # if the number of the values is different from the arity then just return True because
        # the constraint is not concerned, in general it is unlikely to get here because we do the projection
        # but to avoid errors we keep it here
        return True

      if(self.parameters == None):
        if(len(values)>1):
          if(self.isNegation):
            return not self.rel(*values) # we check if the relation accepts the values given to the variables, if so return not the result because it is a negation
          else:
            return self.rel(*values) # we check if the relation accepts the values given to the variables
        else:
          if(self.isNegation):
            return not self.rel(values[0])
          else:
            return self.rel(values[0])
      else:
        if(len(values)>1):
          if(self.isNegation):
            return not self.rel(*values, *self.parameters)
          else:
            return self.rel(*values, *self.parameters)

        else:
          if(len(self.parameters)>1):
            if(self.isNegation):
              return not self.rel(values[0], *self.parameters)
            else:
              return self.rel(values[0], *self.parameters)
          else:
            if(self.isNegation):
              return not self.rel(values[0], self.parameters[0])
            else:
              return self.rel(values[0], self.parameters[0])

    def prepareCpModelConstraint(self, vars, model):
      tmp = []
      for  i in self.scope_ids:
        tmp.append(vars[i])

      if(self.rel == alldiff): #the alldifferent constraint
        if(self.isNegation):
          model += ~ cp.AllDifferent(tmp)
        else:
          model += cp.AllDifferent(tmp)

      elif(self.rel == allequ):
        if(self.isNegation):
          model += ~ cp.AllEqual(tmp)
        else:
          model += cp.AllEqual(tmp)

      elif(self.rel == eqDist1):
        if(self.isNegation):
          model += ~ (abs(tmp[0] - tmp[1]) == 1)
        else:
          model += abs(tmp[0] - tmp[1]) == 1

      elif(self.rel == neDist1):
        if(self.isNegation):
          model += ~(abs(tmp[0] - tmp[1]) != 1)
        else:
          model += abs(tmp[0] - tmp[1]) != 1

      elif(self.rel in [eqConst, neConst , ltConst , leConst, gtConst, geConst]):
        if(self.isNegation):
          model += ~(self.rel(*tmp, *self.parameters))
        else:
          model += self.rel(*tmp, *self.parameters)

      else:
        if(self.parameters!=None):
          if(self.isNegation):
            model += ~(self.rel(*tmp, *self.parameters))
          else:
            model += self.rel(*tmp, *self.parameters)
        else:

          if(self.isNegation):
            model += ~(self.rel(*tmp))  # add the constraint to the model specifying the variables
          else:
            model += self.rel(*tmp)

    def returnCpModelConstraint(self, vars):
      tmp = []
      for  i in self.scope_ids:
        tmp.append(vars[i])

      if(self.rel == alldiff): #the alldifferent constraint
        if(self.isNegation):
          return ~ cp.AllDifferent(tmp)
        else:
          return cp.AllDifferent(tmp)

      elif(self.rel == allequ):
        if(self.isNegation):
          return ~ cp.AllEqual(tmp)
        else:
          return cp.AllEqual(tmp)

      elif(self.rel == eqDist1):
        if(self.isNegation):
          return ~ (abs(tmp[0] - tmp[1]) == 1)
        else:
          return abs(tmp[0] - tmp[1]) == 1

      elif(self.rel == neDist1):
        if(self.isNegation):
          return ~(abs(tmp[0] - tmp[1]) != 1)
        else:
          return abs(tmp[0] - tmp[1]) != 1

      elif(self.rel in [eqConst, neConst , ltConst , leConst, gtConst, geConst]):
        if(self.isNegation):
          return ~(self.rel(*tmp, *self.parameters))
        else:
          return self.rel(*tmp, *self.parameters)

      else:
        if(self.parameters!=None):
          if(self.isNegation):
            return ~(self.rel(*tmp, *self.parameters))
          else:
            return self.rel(*tmp, *self.parameters)
        else:

          if(self.isNegation):
            return ~(self.rel(*tmp))  # add the constraint to the model specifying the variables
          else:
            return self.rel(*tmp)

    # def prepareCpModelConstraintReification(self, vars_dict, model, booll):

    #   tmp = []
    #   for  i in self.scope_ids:
    #     tmp.append(vars_dict[i])

    #   if(self.rel == alldiff): #the alldifferent constraint
    #     if(self.isNegation):
    #       model += (~ cp.AllDifferent(tmp)).implies(booll)
    #       model += booll.implies((~ cp.AllDifferent(tmp)))


    #     else:
    #       model += (cp.AllDifferent(tmp)).implies(booll)
    #       model += booll.implies(cp.AllDifferent(tmp))



    #   elif(self.rel == allequ):
    #     if(self.isNegation):
    #       model +=  (~cp.AllEqual(tmp)).implies(booll)
    #       model +=  booll.implies(~cp.AllEqual(tmp))


    #     else:
    #       model += cp.AllEqual(tmp).implies(booll)
    #       model += booll.implies(cp.AllEqual(tmp))


    #   elif(self.rel == eqDist1):
    #     if(self.isNegation):
    #       model +=  (~(abs(tmp[0] - tmp[1]) == 1)).implies(booll)
    #       model +=  booll.implies(~(abs(tmp[0] - tmp[1]) == 1))


    #     else:
    #       model+= (abs(tmp[0] - tmp[1]) == 1).implies(booll)
    #       model +=booll.implies(abs(tmp[0] - tmp[1]) == 1)



    #   elif(self.rel == neDist1):
    #     if(self.isNegation):
    #       model += (~(abs(tmp[0] - tmp[1]) != 1)).implies(booll)
    #       model += booll.implies(~(abs(tmp[0] - tmp[1]) != 1))


    #     else:
    #       model += (abs(tmp[0] - tmp[1]) != 1).implies(booll)
    #       model += booll.implies(abs(tmp[0] - tmp[1]) != 1)



    #   elif(self.rel in [eqConst, neConst , ltConst , leConst, gtConst, geConst]):
    #     if(self.isNegation):
    #       model += (~(self.rel(*tmp, *self.parameters))).implies(booll)
    #       model += booll.implies(~(self.rel(*tmp, *self.parameters)))


    #     else:
    #       model += (self.rel(*tmp, *self.parameters)).implies(booll)
    #       model += booll.implies(self.rel(*tmp, *self.parameters))



    #   else:
    #     if(self.parameters!=None):
    #       if(self.isNegation):
    #         model += (~(self.rel(*tmp, *self.parameters))).implies(booll)
    #         model += booll.implies(~(self.rel(*tmp, *self.parameters)))


    #       else:
    #         model += (self.rel(*tmp, *self.parameters)).implies(booll)
    #         model += booll.implies(self.rel(*tmp, *self.parameters))


    #     else:
    #       if(self.isNegation):
    #         model += (~(self.rel(*tmp))).implies(booll)
    #         model += booll.implies(~(self.rel(*tmp)))  # add the constraint to the model specifying the variables


    #       else:
    #         model += (self.rel(*tmp)).implies(booll)
    #         model += booll.implies(self.rel(*tmp))


    def prepareCpModelConstraintReification(self, vars_dict, model, booll):

      tmp = []
      for  i in self.scope_ids:
        tmp.append(vars_dict[i])

      if(self.rel == alldiff): #the alldifferent constraint
        if(self.isNegation):
          model += cp.all([(~ cp.AllDifferent(tmp)).implies(booll), booll.implies((~ cp.AllDifferent(tmp)))])


        else:
          model+= cp.all([(cp.AllDifferent(tmp)).implies(booll),booll.implies(cp.AllDifferent(tmp))])



      elif(self.rel == allequ):
        if(self.isNegation):
          model += cp.all([(~cp.AllEqual(tmp)).implies(booll),booll.implies(~cp.AllEqual(tmp))])


        else:
          model+= cp.all([cp.AllEqual(tmp).implies(booll),booll.implies(cp.AllEqual(tmp))])


      elif(self.rel == eqDist1):
        if(self.isNegation):
          model+= cp.all([(~(abs(tmp[0] - tmp[1]) == 1)).implies(booll),booll.implies(~(abs(tmp[0] - tmp[1]) == 1))])


        else:
          model+= cp.all([(abs(tmp[0] - tmp[1]) == 1).implies(booll),booll.implies(abs(tmp[0] - tmp[1]) == 1)])



      elif(self.rel == neDist1):
        if(self.isNegation):
          model += cp.all([(~(abs(tmp[0] - tmp[1]) != 1)).implies(booll),booll.implies(~(abs(tmp[0] - tmp[1]) != 1))])

        else:
          model += cp.all([(abs(tmp[0] - tmp[1]) != 1).implies(booll),booll.implies(abs(tmp[0] - tmp[1]) != 1)])



      elif(self.rel in [eqConst, neConst , ltConst , leConst, gtConst, geConst]):
        if(self.isNegation):
          model+= cp.all([(~(self.rel(*tmp, *self.parameters))).implies(booll),booll.implies(~(self.rel(*tmp, *self.parameters)))])


        else:
          model += cp.all([(self.rel(*tmp, *self.parameters)).implies(booll),booll.implies(self.rel(*tmp, *self.parameters))])



      else:
        if(self.parameters!=None):
          if(self.isNegation):
            model += cp.all([(~(self.rel(*tmp, *self.parameters))).implies(booll),booll.implies(~(self.rel(*tmp, *self.parameters)))])

          else:
            model += cp.all([(self.rel(*tmp, *self.parameters)).implies(booll),booll.implies(self.rel(*tmp, *self.parameters))])


        else:
          if(self.isNegation):
            model += cp.all([(~(self.rel(*tmp))).implies(booll),booll.implies(~(self.rel(*tmp)))])

          else:
            model += cp.all([(self.rel(*tmp)).implies(booll),booll.implies(self.rel(*tmp))])






    def isEqualTo(self, constraint):

      # vars_ids, vars_domains, vars_types Are globals

      # if(not isinstance(constraint, Constraint)):
      #   return False

      if(isinstance(constraint, Conjunction)):
        conjScp = set(constraint.scope)
        if(set(self.scope_ids) != conjScp):
          return False

        v = set(self.scope_ids).union(conjScp)

        # #print("\n The union of the scope: ", v)
        prob_data = {}
        for vv in v:
          prob_data[vv] = problem_data[vv]
          #prob_data[vv]["domain"] = (0,len(v))

        # # print("\n The prob_data: ", prob_data.keys())

        # # print("\n The lens are equal ?: ", len(prob_data.keys())==len(v) )

        n1 = Network(
                 prob_data,
                 [self])

        n2 = Network(
                      prob_data,
                      [constraint])

        e = n1.solve()

        if(not n2.isAccepted(e)):
          return False

        e = n2.solve()

        if(not n1.isAccepted(e)):
          return False

        notself = deepcopy(self)
        notself.isNegation = True

        notconstraint = deepcopy(constraint)
        notconstraint.isNegation = True

        if(n1.addConstraint(notconstraint).solve() ==[] and n2.addConstraint(notself).solve() == [] ):
          return True

        return False


        # # return listOfSolsEqual(n1.getAllSolutions(), n2.getAllSolutions())

        # # elementaryConsts = deepcopy(constraint.elementaryConstraints)
        # notc = deepcopy(self)
        # notc.isNegation = True

        # # # print("\nElementary constraints of the conjunction: \n")
        # # # for c in elementaryConsts:
        # # #   print(c.rel, c.scope_ids)

        # # elementaryConsts.append(notc)
        # # n = Network(prob_data, elementaryConsts)
        # n = Network(prob_data, [constraint,notc])


        # # # print("Notc: \n")
        # # # print(notc.rel, notc.scope_ids, notc.isNegation)

        # # # print(n.solve())
        # # # print(input())
        # if(n.solve()==[]):
        #   return True
        # else:
        #   return False

      if self.id == constraint.id:
        return True
      else:

        conjScp = set(constraint.scope_ids)
        if(set(self.scope_ids) != conjScp):
          return False

        v = set(self.scope_ids).union(conjScp)

        # #print("\n The union of the scope: ", v)
        prob_data = {}
        for vv in v:
          prob_data[vv] = problem_data[vv]
          #prob_data[vv]["domain"] = (0,len(v))

        # # print("\n The prob_data: ", prob_data.keys())

        # # print("\n The lens are equal ?: ", len(prob_data.keys())==len(v) )

        n1 = Network(
                  prob_data,
                  [self])

        n2 = Network(
                      prob_data,
                      [constraint])

        e = n1.solve()

        if(not n2.isAccepted(e)):
          return False

        e = n2.solve()

        if(not n1.isAccepted(e)):
          return False

        notself = deepcopy(self)
        notself.isNegation = True

        notconstraint = deepcopy(constraint)
        notconstraint.isNegation = True

        if(n1.addConstraint(notconstraint).solve() ==[] and n2.addConstraint(notself).solve() == [] ):
          return True

        return False


def listOfSolsEqual(l1,l2):

  if(l1 == [] and l2 == []):
    return True

  if(l1 != [] and l2 == []):
    return False

  if(l1 == [] and l2 != []):
    return False

  if len(l1) != len(l2):
    return False


  cpt1=0
  for a in l1:
    if a in l2:
      cpt1+=1

  cpt2=0
  for a in l2:
    if a in l1:
      cpt2+=1
  if(cpt1 == len(l1) and cpt2 == len(l2)):
    return True

  return False

## Conjunction

In [ ]:
import numpy as np


class Conjunction():

  def __init__(self, constraints, isNegation =False, isRedundant=False):
    # a conjunction is the AND of constraints

    self.scope_ids = [c.scope_ids for c in constraints]

    self.rel = [c.rel for c in constraints]

    self.constraints = constraints

    self.elementaryConstraints = self.AllElementaryConstraints()

    self.scope = self.getScope()

    #self.id = sum([c.id for c in self.constraints])
    self.id = id(self)

    self.isNegation = isNegation
    self.isRedundant= isRedundant

  def getScope(self):

    scopes = []

    for c in self.constraints:
      if isinstance(c,Constraint):
        scopes.append(set(c.scope_ids))
      else:
        scopes.append(set(c.scope))

    ss = list(set.union(*scopes))

    return list(np.unique(ss))


  def AllElementaryConstraints(self):

    elementaryConstraints = []

    for c in self.constraints:
      if isinstance(c,Constraint):
        elementaryConstraints.append(c)
      else:
        elementaryConstraints+= c.elementaryConstraints

    return elementaryConstraints


  def isEqualTo(self, conj):

    # if(not isinstance(conj, Conjunction)):
    #   return False

    if(isinstance(conj, Constraint)):
      return conj.isEqualTo(self)

    if self.id == conj.id:
      return True
    else:
      conjScp = set(conj.scope)
      if(set(self.scope) != conjScp):
        return False

      v = set(self.scope).union(conjScp)

      # #print("\n The union of the scope: ", v)
      prob_data = {}
      for vv in v:
        prob_data[vv] = problem_data[vv]
        #prob_data[vv]["domain"] = (0,len(v))

      # # print("\n The prob_data: ", prob_data.keys())

      # # print("\n The lens are equal ?: ", len(prob_data.keys())==len(v) )

      n1 = Network(
                prob_data,
                [self])

      n2 = Network(
                    prob_data,
                    [conj])

      e = n1.solve()

      if(not n2.isAccepted(e)):
        return False

      e = n2.solve()

      if(not n1.isAccepted(e)):
        return False

      notself = deepcopy(self)
      notself.isNegation = True

      notconstraint = deepcopy(conj)
      notconstraint.isNegation = True

      if(n1.addConstraint(notconstraint).solve() ==[] and n2.addConstraint(notself).solve() == [] ):
        return True

      return False



  def verify(self, assignement,liste = []):
    # for a conjunction to accept an assignement, all constraints from which
    # it is built must accept it (i.e there is an AND between these constraints )

    liste1 = []
    for c in self.constraints:
      if(isinstance(c,Constraint)):
        liste1.append(c.verify(assignement))

    liste2 = []
    for c in self.constraints:
      if(not isinstance(c,Constraint)):
        liste2.append(c.verify(assignement,liste2))

    listtt = liste1 + liste2
    if (listtt == []):
      return all([listtt]) # if only one constraint doesn't accept the assignement this returns False. It returns True otherwise
    else:
      res = all(listtt)
      listtt = []
      return res

  def prepareCpModelConstraint(self, vars_dict, model):

    # This method adds the constraints in this conjunction to the model
    if not self.isNegation:
      ec = deepcopy(self.elementaryConstraints)
      for c in ec:
        model+= cp.all([c.returnCpModelConstraint(vars_dict) for c in ec])
    else:
      ec = deepcopy(self.elementaryConstraints)
      for c in ec:
        c.isNegation = True
      model+= cp.any([c.returnCpModelConstraint(vars_dict) for c in ec])

  # def prepareCpModelConstraintReification(self,vars_dict,model,booll):
  #   model+= (cp.all([c.returnCpModelConstraint(vars_dict) for c in self.elementaryConstraints])).implies(booll)
  #   model+= booll.implies(cp.all([c.returnCpModelConstraint(vars_dict) for c in self.elementaryConstraints]))

  def prepareCpModelConstraintReification(self,vars_dict,model,booll):
    model += cp.all([(cp.all([c.returnCpModelConstraint(vars_dict) for c in self.elementaryConstraints])).implies(booll),booll.implies(cp.all([c.returnCpModelConstraint(vars_dict) for c in self.elementaryConstraints])) ])



  def isScopeIncludedInY(self,Y, liste = []):
  # this method verifies if the scope is included in Y
  # the scope of a conjunction is all the variables in the scopes of the constraints from which it is built

    scope = self.scope

    if set(scope).issubset(set(Y)):
      return True

    return False


  def isScopeIsExactlyY(self,Y):

    # this method verifies if the scope is Exactly Y
    scope = self.scope
    return set(scope) == set(Y)


In [ ]:
# problem_data = {
#     1:{
#       "domain":(1,5),
#        "type":"d",
#        "name":"A"
#     },
#     2:{
#         "domain":(1,5),
#        "type":"d",
#        "name":"B"
#     },
#     3:{
#         "domain":(1,5),
#        "type":"d",
#        "name":"C"
#     },
#     4:{
#         "domain":(1,5),
#        "type":"d",
#        "name":"D"
#     },
#     5:{
#         "domain":(1,5),
#        "type":"d",
#        "name":"E"
#     },
# }


# # c = Conjunction([
# #     Constraint([1,2], operator.eq, 2, True),
# #     Constraint([1,3], operator.eq, 2, True),
# #     Conjunction([
# #       Constraint([1,4], operator.eq, 2, True),
# #       Constraint([1,5], operator.eq, 2, True),
# #     ])
# # ])

# # n1 = Network(problem_data, [c])
# # print("before: ", n1.solve())

# # c.isNegation = True
# # n2 = Network(problem_data, [c])
# # print("after: ", n2.solve())



# c = Conjunction([
#     Constraint([1,3], operator.ne, 2, True),
#     Constraint([1,3], operator.ge, 2, False),
# ])



# co = Constraint([1,3], operator.gt, 2, False)


# co.isEqualTo(c)

## Network

In [ ]:
from ortools.sat.python import cp_model
from itertools import permutations, combinations

# This was borrowed from ortools official documentation, it catches all the solutions
class VarArraySolutionPrinter(cp_model.CpSolverSolutionCallback):
    """Print intermediate solutions."""

    def __init__(self, variables, Limit):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        self.__solution_count = 0
        self.__solutions = []
        self.Limit = Limit

    def on_solution_callback(self):
        self.__solution_count += 1

        e = []
        self.__solution_count+=1
        for v in self.__variables.keys():
          e.append(
              (v, self.__variables[v].Name() , self.Value(self.__variables[v]))
          )
        #print(f"Solution {self.__solution_count}: ",e)

        self.__solutions.append(e)

        if(self.Limit != None):
          if(self.__solution_count > self.Limit):
             self.StopSearch()

    def solution_count(self):
        return self.__solution_count

    def allSolutions(self):
      return self.__solutions

In [ ]:
class Network():

  def __init__(self, problem_data ,constraints):

    self.problem_data = problem_data

    self.constraints = constraints

    self.all_solutions = []

  def isAccepted(self, e):
    # An assignement e is accepted by a network if it doesn't violate any constraints of the network
    violated = None
    for c in self.constraints:
      if not c.verify(e) :
        violated = c

    return all( [c.verify(e) for c in self.constraints]), violated

  def isSolution(self,e):
    # e is a solution if it is complete and accepted
    ids = [ee[0] for ee in e]
    return self.isAccepted(e) and set(ids) == set(self.problem_data.keys())


  def ConstraintsIncludedInY(self,Y):
    # This method constructs a network from the constraints
    # that have scope included in Y

    res = []

    for c in self.constraints:
      if(isinstance(c,Constraint)):
        if set(c.scope_ids).issubset(set(Y)):
            res.append(c)
      else:
        if(c.isScopeIncludedInY(Y)):
          res.append(c)

    prob_data = {}
    for y in Y:
      prob_data[y] = self.problem_data[y]


    return Network(prob_data,res)


  def ConstraintsIsExactlyY(self,Y):
    # This method constructs a network from the constraints
    # that have scope is Exactly Y

    res = []

    for c in self.constraints:
      if(isinstance(c,Constraint)):
        if set(c.scope_ids) == set(Y) :
          res.append(c)
      else:
        if(c.isScopeIsExactlyY(Y)):
          res.append(c)


    prob_data = {}
    for y in Y:
      prob_data[y] = self.problem_data[y]


    return Network(prob_data,res)


  def solve(self):

    model1 = cp.Model()

    vars = {id: cp.intvar(self.problem_data[id]["domain"][0],self.problem_data[id]["domain"][1], shape=1, name=self.problem_data[id]["name"] )  for id in self.problem_data.keys()}

    for c in self.constraints:
      c.prepareCpModelConstraint(vars,model1)

    for v in vars.keys():
        model1 += vars[v] >= vars[v].lb  # this is necessary because cpmpy doesn't instantiate vars that are not used in any constraint


    if(model1.solve(solver="exact")):
      e = []
      for v in vars.keys():
        e.append(
            (v, vars[v].name, vars[v].value())
          )
      return e
    else:
        #print('No solution found.')
        return []


  def allSolutions(self, TimeLimit = None, SolutionLimit= None):
    self.all_solutions = []

    model2 = cp.Model()

    vars = {id: cp.intvar(self.problem_data[id]["domain"][0],self.problem_data[id]["domain"][1], shape=1, name=self.problem_data[id]["name"] )  for id in self.problem_data.keys()}


    for c in self.constraints:
      c.prepareCpModelConstraint(vars,model2)

    for v in vars.keys():
        model2 += vars[v] >= vars[v].lb

    model2.solveAll(solver="exact",display = lambda : self.all_solutions.append([(k,vars[k].name, vars[k].value()) for k in vars.keys()]), time_limit= TimeLimit, solution_limit = SolutionLimit)

  def getAllSolutions(self, TimeLimit = None, SolutionLimit= None):
    # this is the function which will be used in other places.
    self.allSolutions(TimeLimit, SolutionLimit)

    return self.all_solutions

  # def isEquivalentTo(self, N):
  #   # If the sets of solutions of two networks T and T2 are equal, then these two networks are equivalent
  #   e1 = self.solve()
  #   if(not N.isAccepted(e1)):
  #     return False

  #   e2 = N.solve()
  #   if(not self.isAccepted(e2)):
  #     return False

  #   selfsols = self.getAllSolutions()
  #   for solution in selfsols:
  #     if not N.isAccepted(solution):
  #       return False

  #   return True



  def networkOfConstraintsThatRejectE(self,e):
    # Here we construct a network containing constraints that reject e
    tmp = []

    Y = set()
    for c in self.constraints:
      if not c.verify(e):
        tmp.append(c)

        if(isinstance(c,Conjunction)):
          Y = Y.union(set(c.scope))
        else:
          Y = Y.union(set(c.scope_ids))




    prob_data = {}
    for y in Y:
      prob_data[y] = self.problem_data[y]



    return Network(prob_data, tmp)


  def removeConstraintsThatRejectE(self,e):
    # we remove the constraints that reject e from the current network
    tmp = []

    for c in self.constraints:
      if c.verify(e):
        tmp.append(c)

    self.constraints = tmp
    return self


  def addConstraint(self,c):
    # we add  the constraint c to the network
    self.constraints.append(c)
    return self

  def removeListOfConstraints(self, l):
    # we remove a list l of constraints from the current network

    tmp = []
    for c in self.constraints:
      #if c not in l:
      if not constraintinConstraints(c,l):
        tmp.append(c)

    self.constraints = tmp

    return self

  def removeRedundants(self):
    tmp = []
    for c in self.constraints:
      #if c not in l:
      if not c.isRedundant:
        tmp.append(c)

    self.constraints = tmp

    return self


  def isEqualTo(self,constraint1, constraint):

    if(not isinstance(constraint, Constraint)):
        return False

    if(constraint1.arity == constraint.arity and  constraint1.parameters == constraint.parameters and constraint1.rel == constraint.rel):

      if(constraint1.scope_ids == constraint.scope_ids):
        return True

      else:
        if(set(constraint1.scope_ids) == set(constraint.scope_ids)):

          if(constraint1.isCommutative and constraint.isCommutative):
            return True


          v = set(constraint1.scope_ids).union(set(constraint.scope_ids))


          prob_data = {}
          for vv in v:
            prob_data[vv] = problem_data[vv]


          n1 = Network(
                    prob_data,
                    [constraint1])

          n2 = Network(
                    prob_data,
                    [constraint])

          return listOfSolsEqual(n1.getAllSolutions(), n2.getAllSolutions())

    return False



  def removeDuplicates(self):
    res = []
    tmp = []

    for c in self.constraints:
      if not self.existAlready2(res,c):
        tmp.append(c)

    self.constraints = tmp
    return self


  def existAlready2(self,res,c):
    if(self.ConstraintinListOfConstraints(c, res)):
      return True
    res.append(c)
    return False

  def ConstraintinListOfConstraints(self,a,listOfConsts):
    for l in listOfConsts:
      if(a.isEqualTo(l)):
        return True

    return False

  def isEquivalentTo(self, net):

    a1 = []
    for c in self.constraints:
      a1.append(isImplied(c, net.constraints))

    a2 = []
    for c in net.constraints:
      a2.append(isImplied(c,self.constraints))

    return all(a1) and all(a2)



def listOfSolsEqual(l1,l2):

  if(l1 == [] and l2 == []):
    return True

  if(l1 != [] and l2 == []):
    return False

  if(l1 == [] and l2 != []):
    return False

  if len(l1) != len(l2):
    return False


  cpt1=0
  for a in l1:
    if a in l2:
      cpt1+=1

  cpt2=0
  for a in l2:
    if a in l1:
      cpt2+=1
  if(cpt1 == len(l1) and cpt2 == len(l2)):
    return True

  return False

def constraintinConstraints(c,l):

  for cc in l:
    if c.isEqualTo(cc):
      return True

  return False

def isImplied(c,consts):
  n = []
  n = Network(problem_data, deepcopy(consts))
  notc = deepcopy(c)
  notc.isNegation = True

  n= n.addConstraint(notc)

  if(n.solve()==[]):
    return True

  return False


## Basis

In [ ]:
from itertools import permutations, combinations

#The language is like this = [(rel,arity, isCommutative, parameters)]


class Basis(Network):
  # A Basis is also a network, but its constraints are built from a language

  def __init__(self,problem_data, language):
    super().__init__(problem_data,[])
    self.language = language

  def build(self):

   # we build the set of constraints using permutations of variables of length == arity of relation (operator)
    if((operator.lt,2,False,None) in self.language and (operator.gt,2,False,None) in self.language):
      self.language.remove((operator.gt,2,False,None))

    if((operator.le,2,False,None) in self.language and (operator.ge,2,False,None) in self.language):
      self.language.remove((operator.ge,2,False,None))

    for l in self.language:
      if(l[1] != None): # this is a bounded constraint
        if(l[1] == 1): # the unary constraints with a parameter
          # if(l[0] in [eqConst, neConst,ltConst, leConst, gtConst, geConst]):
          #   for v in self.vars_ids:
          #     for param in [1,2,3,4,5]:
          #       self.constraints.append(Constraint([v],l[0],l[1],l[2],[param]))

          if(l[3] != None):
            for v in self.problem_data.keys():
              for param in l[3]:
                self.constraints.append(Constraint([v],l[0],l[1],l[2],[param]))
        else:
          if(l[2] == False): # Not commutative, then we need both [a,b] and [b,a] so we do permutations
            for p in permutations(self.problem_data.keys(),l[1]):
              self.constraints.append(Constraint(list(p),l[0],l[1],l[2]))
          else: #Commutative, just [a,b] is enough so we use combinations
            if(l[0] in [eqDist, eqDist3, neDist, neDist3]):
              for p in combinations(self.problem_data.keys(),l[1]):
                aa = np.array(list(p))
                if(len(np.unique(aa)) == len(aa)):
                  self.constraints.append(Constraint(list(p),l[0],l[1],l[2]))
            else:
              for p in combinations(self.problem_data.keys(),l[1]):
                self.constraints.append(Constraint(list(p),l[0],l[1],l[2]))


      else: # this is an unbounded constraint(global)
        ll = 2
        while( ll<=len(self.problem_data.keys())):
          for p in permutations(self.problem_data.keys(),ll):
            self.constraints.append(Constraint(list(p),l[0],l[1],l[2]))
          ll+=1

## Send a query

In [ ]:
def ask(e, Target):
  global Qc
  global Qa
  global oldsize
  global newsize
  global maxwaittimeold
  global waittimes
  global Examplesizes



  now = time.time()

  waittimes.append(now-maxwaittimeold)
  maxwaittimeold = now

  Qc+=1
  Examplesizes.append(len(e))

  if(newsize > oldsize):
    oldsize = newsize
    if(Qc > Qa):
      Qa = Qc

  print("Classify this example: \n\n")
  print(e)

  if set([ee[0] for ee in e]) == set(Target.problem_data.keys()):
    # This is a membership query
    #print("Hereeeee 367 : ", e)
    #if e in Target.getAllSolutions(): # not necessary and when we generate just for example 10000 solutions, this will return false if
                                       # e is not in the first 10000 even if it is a solution
    res, cons = Target.isAccepted(e)
    if res: # we don't need all solutions here !!!!!!!!!, just verify if it is accepted or not !
      print("\n a COMPLETE query was sent, the answer was YES")
      #print("The example was : \n\n", e)
      return True, cons
    else:
      print("\n a COMPLETE query was sent, the answer was NO")
      # print("The Negative Example was : \n\n", e)
      # print(input())
      return False, cons

  else:
    # This is a partial query
    #print("Hereeeee 376 : ", e)
    res, cons = Target.isAccepted(e)
    if res:
      print("\n a PARTIAL query was sent, the answer was YES")
      #print("The example was : \n\n", e)
      return True, cons
    else:
      print("\n a PARTIAL query was sent, the answer was NO")
      # print("The Negative Example was : \n\n", e)
      # print(input())
      return False, cons

## Generate Example with cuttoff

In [ ]:
import time


def solve(n, scope):

  st = time.time()
  b = n.ConstraintsIncludedInY(scope)

  r = b.solve()
  if( r == []):
    return [], time.time()-st
  else:
    return projection(r,scope) , time.time()-st
    #return r , time.time()-st

def getVarIdBdeg(X):
  global B
  global vars_ids
  nbConsts = 0
  idd = None
  for x in X:
    cpt = 0
    for c in B.constraints:
      if(isinstance(c, Conjunction)):
        if(x in c.scope):
          cpt+=1
      else:
        if(x in c.scope_ids):
          cpt+=1

    if(cpt>nbConsts):
      # print("\n New max consts: ", cpt)
      # print("\n Var id ", x)
      nbConsts = cpt
      idd = x

  # print("\n Final max consts: ", nbConsts)
  # print("\n Var id ", idd)
  return idd




def solveCuttOffCore(n, scope, remainingTime):
  Y = scope[:]
  best = None
  X = list(set(problem_data.keys()).difference(set(Y)))[:]

  start = time.time()


  while(X != []):



    #Y.append(X[0]) # use bdeg heuristic
    i = getVarIdBdeg(X)
    if(i!=None):
      Y.append(i)
    else:
      Y.append(X[0])
    #print("\n\n Next Y: ", Y)
    X = list(set(problem_data.keys()).difference(set(Y)))
    #print("\n\n Next X: ", X)
    Y = sorted(Y)
    b = n.ConstraintsIncludedInY(Y)

    model3 = cp.Model()

    vars = {id: cp.intvar(b.problem_data[id]["domain"][0],b.problem_data[id]["domain"][1], shape=1, name=b.problem_data[id]["name"] )  for id in b.problem_data.keys()}

    for c in b.constraints:
      c.prepareCpModelConstraint(vars,model3)

    for v in vars.keys():
        model3 += vars[v] >= vars[v].lb  # this is necessary because cpmpy doesn't instantiate vars that are not used in any constraint

    if(model3.solve(solver="exact",time_limit=remainingTime) ):
      s = []
      for v in vars.keys():
        s.append(
            (v, vars[v].name, vars[v].value())
          )
    else:
      if(model3.cpm_status.exitstatus.name == "UNSATISFIABLE"):
        return []
      else:
        return None


      # if time.time() - start > remainingTime:
      #   return best

    for v in X:
      #print("\n\n\n Inside if leads to arc inconsistency, remainingtime was : ",remainingTime)
      #print("\n Inside if leads to arc inconsistency, Time consumed so far : ", time.time() - start)
      #print("\nExit condition : ", time.time() - start > remainingTime)
      # if time.time() - start > remainingTime:
      #   #print("HERE 46 will return None")
      #   return None

      tmp = n.ConstraintsIsExactlyY(Y+[v])
      tmp2 = n.ConstraintsIsExactlyY([v]+Y)

      if(tmp.solve()==[] or tmp2.solve==[]):
        #leads to arc inconsistency
        return []

      # #print("\nExit condition : ", time.time() - start > remainingTime)
      # if time.time() - start > remainingTime:
      #   return None
  #best = projection(s,Y)
    best = s

    # print("\n\n\nRemainingtime was : ",remainingTime)
    # print("\nTime consumed so far : ", time.time() - start)

    # print("\nExit condition : ", time.time() - start > remainingTime)
    if time.time() - start > remainingTime:
      #print("HERE 58 will return None")
      return best
  return best

def solveCuttOff(n,scope, remainingTime):


  started = time.time()
  s = solveCuttOffCore(n,scope, remainingTime)

  #print("\nAfter solveCuttOffCore:\n return value : ", s, " remainingTime: ", remainingTime)
  #print("It took: ", time.time() - started)

  if(s == None):
    return (None, remainingTime)
  else:
    return (s, time.time() - started)




In [ ]:
import time
from copy import deepcopy

redundants = []
def generateExample(X,L, B):
  global redundants
  cuttoff = 1
  timer = 0



  cpt = 0
  while(cpt<len(B.constraints)):
    cc = deepcopy(B.constraints[cpt])
    cc.isNegation = True
    ll = deepcopy(L)
    ll = ll.addConstraint(cc)

    e,t = solve(ll, cc.scope_ids)

    timer+=t

    if e==[]:
      redd = B.constraints[cpt]
      redd.isRedundant = True
      redundants.append(redd)
      L = L.addConstraint(redd)
      B = B.removeListOfConstraints([redd])
    else:
      rt = cuttoff - timer
      if(rt<0):
        rt = 0
      ep, tp = solveCuttOff(ll, cc.scope_ids, rt)
      timer += tp

      if ep == None:
        return e

      if ep == []:
        redd = B.constraints[cpt]
        redd.isRedundant = True
        redundants.append(redd)
        L = L.addConstraint(redd)
        B = B.removeListOfConstraints([redd])
      else:
        return ep
  print("Redundants: \n")
  for r in redundants:
    print(r)

  print("before remove reudndants: \n")
  print(len(L.constraints))
  #L = L.removeListOfConstraints(redundants)
  L = L.removeRedundants()
  print("after remove reudndants: \n")
  print(len(L.constraints))
  return []


## FindScope

In [ ]:
def ConstraintinListOfConstraints(a,listOfConsts):

  for l in listOfConsts:
    if(a.isEqualTo(l)):
      return True

  return False
def areListsOfConstraintsEqual(l1,l2):
  if(l1 == [] and l2!=[]):
    return False
  if(l1 != [] and l2==[]):
    return False
  if(l1 == [] and l2==[]):
    return True
  if(len(l1)!=len(l2)):
    return False

  cpt1  = 0
  for l in l1:
    if ConstraintinListOfConstraints(l,l2):
      cpt1+=1

  cpt2  = 0
  for l in l2:
    if ConstraintinListOfConstraints(l,l1):
      cpt2+=1

  if(cpt1 == len(l1) and cpt2 == len(l2)):
    return True

  return False

def FindScope(e, R ,Y , B):
  global calls
  global Target

  print("\n\nFind scope call -------------------------------------------\n\n")
  # calls+=1

  # print("Inside findscope !!!!!! \n: ")

  print("R: ", [r for r in R])
  print("Y: ", [y for y in Y])

  # if(calls > 20):
  #   print("Possible Recusion Error ") # For test purpose, to be removed later
  #   return

  # print("len(B.networkOfConstraintsThatRejectE(projection(e,R)).constraints): ", B.networkOfConstraintsThatRejectE(projection(e,R)).constraints)
  # for c in B.networkOfConstraintsThatRejectE(projection(e,R)).constraints:
  #   print(c.rel, c.scope_ids)


  # print("the test", B.networkOfConstraintsThatRejectE(projection(e,R)).constraints != [])
  if B.networkOfConstraintsThatRejectE(projection(e,R)).constraints != [] :
    if(ask(projection(e,R), Target)[0]):
      # print("Here")
      # print("In FindScope B[",Y,"] before removal: \n")
      # for c in B.ConstraintsIncludedInY(Y).constraints:
      #   print(c.scope_ids, c.rel)
      # print(projection(e,R)," was classified positive")
      B = B.removeConstraintsThatRejectE(projection(e,R))
      # print("\nInside FindScope B reduced in size, new size: ",len(B.constraints))
      # print("In FindScope B[",Y,"] After removal: \n")
      # for c in B.ConstraintsIncludedInY(Y).constraints:
      #   print(c.scope_ids, c.rel)
    else:
      # print("Here 2")
      # print(projection(e,R)," was classified negative")
      return []

  if len(Y) == 1:
    return Y

  l = len(Y)

  l2 = l//2

  if l == 2 :
    Y1 = Y[:l2]
    Y2 = Y[l2:]
  else:
    Y1 = Y[:l2+1]
    Y2 = Y[l2+1:]


  if(areListsOfConstraintsEqual(B.networkOfConstraintsThatRejectE(projection( e,list(set(R).union(set(Y1))))).constraints , B.networkOfConstraintsThatRejectE(projection( e,list(set(R).union(set(Y))))).constraints ) ):
    #print("Here !!! 1")
    S1 = []
  else:
    #print("Here !!! 2")
    S1 = FindScope(e, list(set(R).union(set(Y1))), Y2, B )

  if( areListsOfConstraintsEqual(B.networkOfConstraintsThatRejectE(projection( e,list(set(R).union(set(S1))))).constraints , B.networkOfConstraintsThatRejectE(projection( e,list(set(R).union(set(Y))))).constraints) ):
    #print("Here !!! 3")
    S2 = []
  else:
    #print("Here !!! 4")
    S2 = FindScope(e, list(set(R).union(set(S1))), Y1, B)


  return list( set(S1).union(set(S2)) )


## Join Networks

In [ ]:
import pandas as pd
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()

def existAlready(res,nc1,nc2):
    # conj = joinConstraints([],nc1,nc2)
    # conj2 = joinConstraints([],nc2,nc1)
    conj = joinConstraints([],nc1,nc2)
    if(ConstraintinListOfConstraints(conj, res)):
        return True
    # if(ConstraintinListOfConstraints(conj2, res)):
    #     return True

    return False



def existAlready2(res,c):

    if(ConstraintinListOfConstraints(c, res)):
        return True
    res.append(c)
    return False


def inListOfLists(a,listOfLists):

  for l in listOfLists:
    if(isinstance(l,int)):
      if(l == a):
        return True
    else:
       res = inListOfLists(a,l)
       if(res):
         return True

  return False


def ConstraintinListOfConstraints(a,listOfConsts):

  for l in listOfConsts:
    if(a.isEqualTo(l)):
      return True

  return False

import time
def areIncompatible(vars_data,c1,c2):


  scope1 = c1.scope_ids if isinstance(c1,Constraint) else c1.scope

  scope2 = c2.scope_ids if isinstance(c2,Constraint) else c2.scope

  scope = set(scope1).union(set(scope2))

  vd = {}

  for id in vars_data.keys():
    if id in scope:
      vd[id] = vars_data[id]

  n = Network(
      vd,
      [c1,c2])

  # print("\n\nThe network inside areIncompatible: \n")
  # print("VarsData: \n")
  # print(n.problem_data)
  # print("Constraints: \n")

  # for c in n.constraints:
  #   print(c.rel,c.scope_ids)

  # st = time.time()
  if(n.solve() == []):
    # print("To prove incompatibility, it took: ", time.time()-st)
    return True

  # print("To prove incompatibility, it took: ", time.time()-st)
  return False


# def joinConstraints(res,c1,c2):
#   if(c1.isEqualTo(c2)):
#     return c1
#   else:
#     res.append(Conjunction([c1,c2]))
#     return Conjunction([c1,c2])


def joinConstraints(res,c1,c2):
  # you can add incompatibility test here to do just one pass
  if(c1.isEqualTo(c2)):
    return c1
  else:
    if(isinstance(c2,Conjunction) and isinstance(c1,Constraint)):
      if(not ConstraintinListOfConstraints(c1, c2.elementaryConstraints)):
        res.append(Conjunction([c1,c2]))
        return Conjunction([c1,c2])
      else:
        res.append(c2)
        return c2
    elif(isinstance(c1,Conjunction) and isinstance(c2,Constraint)):
      if(not ConstraintinListOfConstraints(c2, c1.elementaryConstraints)):
        res.append(Conjunction([c1,c2]))
        return Conjunction([c1,c2])
      else:
        res.append(c1)
        return c1
    elif(isinstance(c1,Conjunction) and isinstance(c2,Conjunction)):
      if(ConstraintinListOfConstraints(c1, c2.constraints)):
        res.append(c2)
        return c2
      elif(ConstraintinListOfConstraints(c2, c1.constraints)):
        res.append(c1)
        return c1
      else:
        res.append(Conjunction([c1,c2]))
        return Conjunction([c1,c2])
    else:
      res.append(Conjunction([c1,c2]))
      return Conjunction([c1,c2])




def joinNetworks(N1,N2):


  if len(N2.constraints) == 0:
    return N1



  #vars_data = [(i,d,t) for i,d,t in zip(N1.vars_ids,N1.vars_domains,N1.vars_types)]

  vars_data = N1.problem_data

  already_ids = [k for k in vars_data.keys()]

  for id in N2.problem_data.keys():
    if(id not in already_ids):
      vars_data[id] = N2.problem_data[id]

  res = []

  df1 = pd.DataFrame({"consts": N1.constraints})
  df2 = pd.DataFrame({"consts": N2.constraints})



  dfr = df1.join(df2, lsuffix='_df1', rsuffix='_df2', how="cross")

  #print("len(delta):\n", len(dfr))
  #tqdm.pandas()
  res = []



  #dfr["Conjunction"] = dfr.progress_apply(lambda x: joinConstraints(res,x.consts_df1, x.consts_df2) if not existAlready(res,x.consts_df1, x.consts_df2) else "eee", axis=1)
  dfr["Conjunction"] = dfr.apply(lambda x: joinConstraints(res,x.consts_df1, x.consts_df2) if not existAlready(res,x.consts_df1, x.consts_df2) and not areIncompatible(vars_data,x.consts_df1, x.consts_df2) else "eee", axis=1)


  dfr = dfr[dfr["Conjunction"] != "eee"]

  #dfr["AreIncompatible"] = dfr.progress_apply(lambda x: areIncompatible(vars_data,x.consts_df1, x.consts_df2), axis=1)
  # dfr["AreIncompatible"] = dfr.apply(lambda x: areIncompatible(vars_data,x.consts_df1, x.consts_df2), axis=1)


  # dfr = dfr[dfr["AreIncompatible"] == False]

  res = []
  #dfr["Conjunction_unique"] = dfr["Conjunction"].progress_apply(lambda x: x if not existAlready2(res,x) else "eee")
  dfr["Conjunction_unique"] = dfr["Conjunction"].apply(lambda x: x if not existAlready2(res,x) else "eee")

  dfr = dfr[dfr["Conjunction_unique"] != "eee"]

  return Network(vars_data,dfr["Conjunction_unique"].values)


<ipython-input-15-4d80adbd618e>:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.*` instead of `tqdm._tqdm_notebook.*`
  from tqdm._tqdm_notebook import tqdm_notebook


In [ ]:
# def joinNetworks(n1,n2):
#   if(len(n1.constraints)==0):
#     return n2

#   if(len(n2.constraints)==0):
#     return n1


#   vars_data = n1.problem_data

#   already_ids = [k for k in vars_data.keys()]

#   for id in n2.problem_data.keys():
#     if(id not in already_ids):
#       vars_data[id] = n2.problem_data[id]

#   resConjs = []
#   resElementary = []

#   for c1 in n1.constraints:
#     for c2 in n2.constraints:
#       if(not c1.isEqualTo(c2) and Network(vars_data, [c1,c2]).solve()!=[]):
#         conj = Conjunction([c1,c2])
#         exist = False
#         for co in resConjs:
#           if(co.isEqualTo(conj)):
#             #exist already
#             exist = True
#             break

#         if(not exist):
#           resConjs.append(conj)

#       else:
#         existE = False

#         for e in resElementary:
#           if(e.isEqualTo(c1)):
#             existE = True
#             break

#         if(not existE):
#           implied = False
#           for co in resConjs:
#             notc = deepcopy(c1)
#             notc.isNegation = True
#             if( Network(vars_data, [co, notc]).solve==[]):
#               implied=True
#               break

#           if not implied:
#             resElementary.append(c1)



#   return Network(vars_data, resConjs + resElementary)


## FindEPrime

In [ ]:
# from tqdm import tqdm
# def isListConstsSubSetOtherListConsts(a,b):
#   if(a == []):
#     return True

#   if(a == [] and b== []):
#     return True

#   if(a != [] and b == []):
#     return False

#   if(len(a)>len(b)):
#     return False


#   for aa in tqdm(a, ascii="True", desc="SubList"):
#     cpt = 0
#     for bb in b:
#       if(aa.isEqualTo(bb)):
#         cpt+=1

#     if(cpt==0):
#       return False

#   return True


# from random import random



# # This was borrowed from ortools official documentation, it catches all the solutions
# class SolutionsCallBack(cp.solvers.ortools.OrtSolutionCounter):


#     def __init__(self, solver ,variables, delta):
#         cp.solvers.ortools.OrtSolutionCounter.__init__(self)


#         self.__solution_count = 0
#         self.__solutions = []

#         self.delta = delta
#         self.ds = len(delta.constraints)
#         self._cpm_vars = solver.user_vars
#         self._varmap = solver._varmap

#         self.solver_vars = solver.solver_vars(list(self._cpm_vars))

#         self.__variables = variables

#         self.variables2 = {}

#         for k in self.__variables.keys():
#           for v in self.solver_vars:
#             if self.__variables[k].name == v.Name():
#               self.variables2[k] = v


#         self.solver = solver


#         self.example = None



#     def on_solution_callback(self):
#         super().on_solution_callback()
#         self.__solution_count+=1

#         e = []
#         for v in self.variables2.keys():
#           e.append(
#               (v, self.variables2[v].Name() , self.Value(self.variables2[v]))
#           )

#         print("\n Inside Find Eprime Solution nb: ", self.solution_count())

#         d = self.delta.networkOfConstraintsThatRejectE(e).constraints

#         if( d!=[] and len(d)!= self.ds ):
#           self.example = e
#           self.StopSearch()

#         # print("solution nb ",self.solution_count()," : ",e)

#     def solution_count(self):
#         return self.__solution_count

#     def allSolutions(self):
#       return self.__solutions

# # def deltaIsImpliedByLY(delta,ll):


# #   cpt=0
# #   while(cpt<len(delta.constraints)):
# #     notc = deepcopy(delta.constraints[cpt])
# #     notc.isNegation = True

# #     lll = deepcopy(ll)

# #     lll = lll.addConstraint(notc)

# #     e = lll.solve()

# #     if e == []:
# #       cpt+=1
# #     else:
# #       return False

# #   return True


# def findEPrime(L,Y,delta):

#   ds = len(delta.constraints)

#   if(ds==0 or ds==1):
#     return [], None, None


#   ll = L.ConstraintsIncludedInY(Y)

#   # if deltaIsImpliedByLY(delta,ll):
#   #   return []


#   model = cp.Model()

#   #vars = { id:  cp.intvar(dom[0],dom[1], shape=1, name=t+str(id))  for id,dom,t in zip(ll.vars_ids, ll.vars_domains, ll.vars_types) }
#   vars = { id:  cp.intvar(ll.problem_data[id]["domain"][0],ll.problem_data[id]["domain"][1], shape=1, name=ll.problem_data[id]["name"])  for id in ll.problem_data.keys()}
#   for c in ll.constraints:
#     c.prepareCpModelConstraint(vars,model)

#   for v in vars.keys():
#       model += vars[v] >= vars[v].lb

#   s = cp.SolverLookup.get("ortools", model) # faster on a solver interface directly
#   cb = SolutionsCallBack(s, vars, delta)

#   s.solve(time_limit=20,enumerate_all_solutions= True, solution_callback=cb)

#   if(cb.example != None):
#     return cb.example, ll, delta
#   else:
#     return findEPrimeReification(L,Y,delta)


In [ ]:
# def solve2(ll,scope):

#   #b = ll.ConstraintsIncludedInY(scope)
#   r = ll.solve()
#   if( r == []):
#     return []
#   else:
#     #return projection(r,scope)
#     return r

# redundants2 = []
# def findEPrime(L,Y,d):

#   delta = deepcopy(d)
#   ll = deepcopy(L.ConstraintsIncludedInY(Y))


#   while(len(delta.constraints)!=0):
#     notc = deepcopy(delta.constraints[0])

#     notc.isNegation = True

#     ll = ll.addConstraint(notc)
#     res = solve2(ll, notc.scope_ids ) if isinstance(notc, Constraint) else solve2(ll, notc.scope )

#     if(res == []):
#       redundants2.append(delta.constraints[0])
#       ll = ll.addConstraint(delta.constraints[0])
#       delta = delta.removeListOfConstraints([delta.constraints[0]])
#     else:
#       if(len(d.networkOfConstraintsThatRejectE(res).constraints)== 0 or len(d.networkOfConstraintsThatRejectE(res).constraints)==len(d.constraints)):
#         return []
#       else:
#         return res


#   ll = ll.removeListOfConstraints(redundants2)
#   return []





In [ ]:
from copy import deepcopy
findEPrimeCall = 0

#def findEPrimeReification(L,Y,d):
def findEPrime(L,Y,d):
  ds = len(d.constraints)

  if(ds==0 or ds==1):
    return [], None, None


  C = deepcopy(L.ConstraintsIncludedInY(Y))

  vars = { id:  cp.intvar(C.problem_data[id]["domain"][0],C.problem_data[id]["domain"][1], shape=1, name=C.problem_data[id]["name"])  for id in C.problem_data.keys() }

  model4 = cp.Model()


  bools = [cp.boolvar(name="bool"+str(i+1)) for i in range(len(d.constraints))]



  elementary = [c for c in d.constraints if isinstance(c,Constraint)]
  Conjs = [c for c in d.constraints if isinstance(c,Conjunction)]

  cpt = 0
  for c in elementary:
    c.prepareCpModelConstraintReification(vars, model4, bools[cpt])
    cpt+=1

  for c in Conjs:
    c.prepareCpModelConstraintReification(vars, model4, bools[cpt])
    cpt+=1

  #objvar = cp.intvar(1, len(bools)-1, shape=1, name="obj")

  #model += objvar >= objvar.lb
  model4 += cp.all([(sum(bools) >= 1),(sum(bools) < len(d.constraints))])



  for c in C.constraints:
    c.prepareCpModelConstraint(vars,model4)

  for v in vars.keys():
      model4 += vars[v] >= vars[v].lb


  #model += sum(bools) != 0

  if(model4.solve(solver="exact")):
      e = []
      for v in vars.keys():
        e.append(
            (v, vars[v].name, vars[v].value())
          )
      return projection(e, Y), C, d
  else:
    return [], C, d

In [ ]:
# import numpy as np

# n = 4
# ub = n * n + 1

# vars_ids =   [i+1 for i in range(n)]
# vars_names = ["m"+str(i+1) for i in range(n)]
# vars_types = ["marker" for i in range(n)]
# vars_domains = [(0,0)]
# vars_domains = vars_domains + [(1,ub) for i in range(n-1)]
# types = ["marker"]


# problem_data = {i+1: {"domain": (1,ub), "name": "m"+str(i+1),"type": "marker"} for i in range(n)}
# problem_data[1]["domain"] = (0,0)
# print(problem_data)






# def eqDist(a,b,c,d):
#   return operator.eq( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))

# def neDist(a,b,c,d):
#   return operator.ne( operator.abs(operator.sub(a,b)) ,  operator.abs(operator.sub(c,d)))


# def eqDist3(repeat,b,c):
#   return operator.eq( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))

# def neDist3(repeat,b,c):
#   return operator.ne( operator.abs(operator.sub(repeat,b)) ,  operator.abs(operator.sub(c,repeat)))


# language = [
#     (operator.ge, 2, False,None),
#     (operator.le, 2, False,None),
#     (operator.gt, 2, False,None),
#     (operator.lt, 2, False,None),
#     (operator.ne, 2, True,None),
#     (operator.eq, 2, True,None),
#     (eqDist, 4, True,None),
#     (neDist, 4, True,None),
#     (eqDist3, 3, False,None),
#     (neDist3, 3, False,None)
# ]

# B = Basis(problem_data , language)
# B.build()
# # B = B.removeDuplicates()

# targetconstraints = []


# # there is an increasing constraint between markers
# for i in range(n-1):
#   targetconstraints.append(Constraint([vars_ids[i],vars_ids[i+1]], operator.lt, 2,False))


# distances = list(combinations(range(1,n+1), 2))


# for i in range(len(distances)-1):
#   for j in range(i+1,len(distances)):
#     p = list(distances[i])+list(distances[j])
#     pp = np.array(p)
#     if(len(np.unique(pp)) == len(pp)):
#       targetconstraints.append(Constraint(p, neDist , 4,True))
#     else:
#       if(p[0]==p[2]):
#         targetconstraints.append(Constraint([p[0],p[1],p[3]], neDist3 , 3,False))
#       elif(p[0]==p[3]):
#         targetconstraints.append(Constraint([p[0],p[1],p[2]], neDist3 , 3,False))
#       elif(p[1]==p[2]):
#         targetconstraints.append(Constraint([p[1],p[0],p[3]], neDist3 , 3,False))
#       elif(p[1]==p[3]):
#         targetconstraints.append(Constraint([p[1],p[0],p[2]], neDist3 , 3,False))



# Target = Network( problem_data, targetconstraints)

In [ ]:
# findEPrime(Target, [1,2,3], B)

In [ ]:
#n1 = Network(problem_data,B.ConstraintsIncludedInY([1,2,3]))
# n2 = Network(problem_data,B.ConstraintsIncludedInY([1,2,3]).constraints)

#nn = joinNetworks(n1,n2)

# for c in nn.constraints:
#   print(c.rel, c.scope_ids)

In [ ]:
# findEPrime(Target, [1,2,3], n2)

## FindC

In [ ]:
def ConstraintinListOfConstraints(a,listOfConsts):

  for l in listOfConsts:
    if(a.isEqualTo(l)):
      return True

  return False


def existAlready2(res,c):

    if(ConstraintinListOfConstraints(c, res)):
        return True
    res.append(c)
    return False

def removeDuplicates(listOfConstraints):
  res = []
  nonDup = []
  for c in listOfConstraints:
    if(existAlready2(res,c)):
      continue
    else:
      nonDup.append(c)

  # print("\nBefore remove inside function: ", len(listOfConstraints))
  # print("\nAfter remove inside function: ", len(nonDup))
  return nonDup

def FindC(e,Y,L,B, TimeLimit = None, SolutionLimit = None):
  global cpt
  global newsize
  global oldsizeexamples
  global newsizeexamples
  global nbExamplesc
  global nbExamplesa
  global Exampletimes
  global byQuAcq
  global newsizenlpqueries1
  global newsizenlpqueries2
  global newsizenlpqueries3
  global nbConfirmationQueriesC
  global nbConfirmationQueriesA
  global oldsizenlpqueries2
  global byNLP
  global Bbk


  delta = []

  delta = B.ConstraintsIsExactlyY(Y)

  print("Inside FindC-----------------------------------: \n\n\n")
  print("e: ", e)
  print("\nY: \n",Y,"\n   B.ConstraintsIsExactlyY(Y): ",[(c.rel,c.scope_ids) for c in B.ConstraintsIsExactlyY(Y).constraints])
  tmpp = delta.networkOfConstraintsThatRejectE(e)
  print("\n  delta.networkOfConstraintsThatRejectE(e): ",[(c.rel,c.scope_ids) for c in delta.networkOfConstraintsThatRejectE(e).constraints])
  # tmpp.constraints = removeDuplicates(tmpp.constraints)

  delta = joinNetworks( delta , tmpp)
  print("\n  delta: ",[(c.rel,c.scope_ids) for c in delta.constraints])

  global findEPrimeCall
  while(True):

    # print("\nSTARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall: ",findEPrimeCall,"---------------------\n\n\n")
    # print("\n\nDelta before findEPrime: \n")
    # for c in delta.constraints:
    #   print(c.rel, c.scope_ids)


    # print("\n\nL[Y] before findEPrime: \n")
    # for c in L.ConstraintsIncludedInY(Y).constraints:
    #   print(c.rel, c.scope_ids)


    stg = time.time()
    ep, ll, d = findEPrime(L,Y,delta)
    #register example sizes and times to generate them
    Exampletimes.append(time.time()-stg)

    #calculate number of examples generated until convergence and until finding an equivalent network
    nbExamplesc+=1
    if(newsizeexamples > oldsizeexamples):
      oldsizeexamples = newsizeexamples
      if(nbExamplesc > nbExamplesa):
        nbExamplesa = nbExamplesc


    # print("\n Example: ", ep)

    # print("\n\nDelta returned from findEPrime: \n")
    # for c in delta.constraints:
    #   print(c.rel, c.scope_ids)


    # print("\n\nL[Y] returned from findEPrime: \n")
    # for c in L.ConstraintsIncludedInY(Y).constraints:
    #   print(c.rel, c.scope_ids)

    # print("\nENDENDENDENDENDENDENDENDENDENDEND findEPrimeCall: ",findEPrimeCall,"---------------------\n\n\n")
    findEPrimeCall+=1
    cadded = None
    if ep == []:
      #print("\n A new constraint was added to the being learned network !!!:\n")

      print("I found a list of possible constraints, now I will ask you to choose one that is realy in the set of constraints you have in your mind\n\n")

      elementary = [c for c in delta.constraints if isinstance(c,Constraint)]
      Conjs = [c for c in delta.constraints if isinstance(c,Conjunction)]

      # print("The candidates!!!!!!!!!!!!!!!!!: \n")
      # for c in delta.constraints:
      #   if isinstance(c,Constraint):
      #     print(c.rel,c.scope_ids,c.isCommutative,c.parameters,c.arity)
      #   else:
      #     print(c.rel, c.scope_ids)

      # print("The violated constraint!!!!!!!!!!!!!!: \n")
      # print(cons.rel,cons.scope_ids,cons.isCommutative,cons.parameters,cons.arity)


      if(len(elementary) != 0):
        for c in elementary:

          nbConfirmationQueriesC += 1

          if(newsizenlpqueries2 > oldsizenlpqueries2):
            oldsizenlpqueries2 = newsizenlpqueries2
            if(nbConfirmationQueriesC > nbConfirmationQueriesA):
              nbConfirmationQueriesA = nbConfirmationQueriesC

          print("Is this a part of what you are looking for ?:\n", c.rel,c.scope_ids,c.isCommutative,c.parameters,c.arity)
          if ConstraintinListOfConstraints(c, Target.constraints):
          #if c.rel == cons.rel and c.scope_ids == cons.scope_ids and c.parameters == cons.parameters and c.isCommutative == cons.isCommutative and c.arity == cons.arity:
            cadded = c
            break
          else:
            continue
      else:
        for c in Conjs:
          nbConfirmationQueriesC += 1

          if(newsizenlpqueries2 > oldsizenlpqueries2):
            oldsizenlpqueries2 = newsizenlpqueries2
            if(nbConfirmationQueriesC > nbConfirmationQueriesA):
              nbConfirmationQueriesA = nbConfirmationQueriesC

          print("Is this a part of what you are looking for ?:\n", c.rel,c.scope_ids)
          if ConstraintinListOfConstraints(c, Target.constraints):
          #if c.rel == cons.rel and c.scope_ids == cons.scope_ids and c.parameters == cons.parameters and c.isCommutative == cons.isCommutative and c.arity == cons.arity:
            cadded = c
            break
          else:
            continue

      #c = delta.constraints[0]

      if cadded != None:
        print("the quacq added constraint: \n")
        print(cadded.rel,cadded.scope_ids)
        L = L.addConstraint(cadded)
        byQuAcq+=1
        newsize+=1
        newsizeexamples+=1
        newsizenlpqueries1+=1
        newsizenlpqueries2+=1
        newsizenlpqueries3+=1

        # print("\n Number of constraints so far: \n", len(L.constraints))
        B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(Y).constraints)
        # print("\n FindC : Remaining constraints in the Basis: \n", len(B.constraints))
        # print(input())
        break
      else:
        print('Here!!!!!!!!! RegularQuAcq2')
        L,scope_ids = RegularQuAcq2(Bbk,L)
        if scope_ids != None:
          B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(scope_ids).constraints)
        else:
          break

    else:
      res, cons2 = ask(ep,Target)
      if(res):
        print("inside findc, ep was classified positive")
        delta = delta.removeConstraintsThatRejectE(ep)
        B = B.removeConstraintsThatRejectE(ep)
        # print("\n FindC : B reduced in size, new size: ",len(B.constraints))
      else:
        res, scope_ids = conversation(ep, cons2)
        if res == "FindC":
          S = FindScope(ep,[],Y,B)
          if set(S).issubset(set(Y)) and len(S)!=len(Y) : #and S!=[]:
            FindC(ep,S,L,B)
          else:
            tmp1 = delta
            tmp11 = tmp1.networkOfConstraintsThatRejectE(ep)
            delta = joinNetworks(tmp1, tmp11)
            print("new delta: ", [(c.rel,c.scope_ids) for c in delta.constraints])
        else:
          if set(scope_ids).issubset(set(Y)) and len(scope_ids)!=len(Y) : #and S!=[]:
            L = L.addConstraint(res)
            print("The nlp added constraint: \n")
            print(res.rel, res.scope_ids)
            print("!!!!!!!!!!!!!!!!!!!!!!!!!\n\n")
            byNLP+=1
            newsize+=1
            newsizeexamples+=1
            newsizenlpqueries1+=1
            newsizenlpqueries2+=1
            newsizenlpqueries3+=1
            B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(scope_ids).constraints)
            print("L after nlp: ", len(L.constraints))
            break
          else:
            tmp1 = delta
            tmp11 = tmp1.networkOfConstraintsThatRejectE(ep)
            delta = joinNetworks(tmp1, tmp11)



In [ ]:
# def FindC(e,Y,L,B, TimeLimit = None, SolutionLimit = None):
#   global cpt

#   delta = []

#   delta = B.ConstraintsIsExactlyY(Y)

#   tmpp = delta.networkOfConstraintsThatRejectE(e)

#   #tmpp.constraints = removeDuplicates(tmpp.constraints)

#   delta = joinNetworks( delta , tmpp)

#   print("\n\n\n-------len(delta) before loop: ", len(delta.constraints))

#   while(True):

#     ep = findEPrime(L,Y,delta)

#     if ep == []:
#       print("\n A new constraint was added to the being learned network !!!:\n")
#       elementary = [c for c in delta.constraints if isinstance(c,Constraint)]
#       Conjs = [c for c in delta.constraints if isinstance(c,Conjunction)]

#       if(len(elementary) != 0):
#         c = elementary[0]
#       else:
#         min = 100000
#         c = None
#         for cc in Conjs:
#           if(len(cc.elementaryConstraints)<min):
#             min = len(cc.elementaryConstraints)
#             c = cc
#       #c = delta.constraints[0]


#       L = L.addConstraint(c)
#       print("\n Number of constraints so far: \n", len(L.constraints))
#       B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(Y).constraints)
#       print("\n FindC : Remaining constraints in the Basis: \n", len(B.constraints))
#       # print(input())
#       break
#     else:
#       if(ask(ep,Target)):
#         delta = delta.removeConstraintsThatRejectE(ep)

#         B = B.removeConstraintsThatRejectE(ep)

#         print("\n FindC : B reduced in size, new size: ",len(B.constraints))
#       else:

#         S = FindScope(ep,[],Y,B)
#         if set(S).issubset(set(Y)) and len(S)!=len(Y) : #and S!=[]:
#           FindC(ep,S,L,B,TimeLimit, SolutionLimit)
#         else:
#           tmp1 = delta
#           tmp11 = tmp1.networkOfConstraintsThatRejectE(ep)
#           delta = joinNetworks(tmp1, tmp11)

#           print("\nAfter join findcloop: ")
#           print(len(delta.constraints))

# Classification

In [ ]:
!gdown 1oJPm8450UhQLCKLK8hw7JGAFPFAzaHn0

Downloading...
From: https://drive.google.com/uc?id=1oJPm8450UhQLCKLK8hw7JGAFPFAzaHn0
To: /content/classif23classes.ckpt
100% 1.31G/1.31G [00:07<00:00, 172MB/s]


In [ ]:
#REFERENCE: https://towardsdatascience.com/fine-tuning-bert-for-text-classification-54e7df642894

In [ ]:
!pip install transformers==4.30.2

import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from tabulate import tabulate
from tqdm import trange
import random


tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
    )


def preprocessing(input_text, tokenizer):
  '''
  Returns <class transformers.tokenization_utils_base.BatchEncoding> with the following fields:
    - input_ids: list of token ids
    - token_type_ids: list of token type ids
    - attention_mask: list of indices (0,1) specifying which tokens should considered by the model (return_attention_mask = True).
  '''
  return tokenizer.encode_plus(
                        input_text,
                        add_special_tokens = True,
                        max_length = 128,
                        pad_to_max_length = True,
                        return_attention_mask = True,
                        return_tensors = 'pt'
                   )

def loadClassifModel(modelPath = "/content/classif23classes.ckpt"):
  # Load the BertForSequenceClassification model
  model = BertForSequenceClassification.from_pretrained(
      'bert-base-uncased',
      num_labels = 24,
      output_attentions = False,
      output_hidden_states = False,
  )

  # Recommended learning rates (Adam): 5e-5, 3e-5, 2e-5. See: https://arxiv.org/pdf/1810.04805.pdf
  optimizer = torch.optim.AdamW(model.parameters(),
                                lr = 5e-5,
                                eps = 1e-08
                                )

  # Run on GPU
  model.cuda()

  checkpoint = torch.load(modelPath)
  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  epoch = checkpoint['epoch']
  loss = checkpoint['loss']

  model.eval()

  return model



device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def classify(sentence, classifModel):
  global device
  # We need Token IDs and Attention Mask for inference on the new sentence
  test_ids = []
  test_attention_mask = []

  # Apply the tokenizer
  encoding = preprocessing(sentence, tokenizer)

  # Extract IDs and Attention Mask
  test_ids.append(encoding['input_ids'])
  test_attention_mask.append(encoding['attention_mask'])
  test_ids = torch.cat(test_ids, dim = 0)
  test_attention_mask = torch.cat(test_attention_mask, dim = 0)

  # Forward pass, calculate logit predictions
  with torch.no_grad():
    output = classifModel(test_ids.to(device), token_type_ids = None, attention_mask = test_attention_mask.to(device))

  #prediction = 'ALL_EQUAL' if np.argmax(output.logits.cpu().numpy()).flatten().item() == 1 else 'ALL_DIFFERENT'



  if np.argmax(output.logits.cpu().numpy()).flatten().item() == 0:
    prediction = "BINARY_gt"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 1:
    prediction = "BINARY_lt"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 2:
    prediction = "BINARY_ge"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 3:
    prediction = "BINARY_le"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 4:
    prediction = "BINARY_eq"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 5:
    prediction = "BINARY_ne"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 6:
    prediction = "BINARYDIST_gt"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 7:
    prediction = "BINARYDIST_lt"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 8:
    prediction = "BINARYDIST_ge"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 9:
    prediction = "BINARYDIST_le"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 10:
    prediction = "BINARYDIST_eq"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 11:
    prediction = "BINARYDIST_ne"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 12:
    prediction = "UNARYDIST_gt"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 13:
    prediction = "UNARYDIST_lt"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 14:
    prediction = "UNARYDIST_ge"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 15:
    prediction = "UNARYDIST_le"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 16:
    prediction = "UNARYDIST_eq"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 17:
    prediction = "UNARYDIST_ne"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 18:
    prediction = "UNARY_le"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 19:
    prediction = "UNARY_lt"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 20:
    prediction = "UNARY_ge"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 21:
    prediction = "UNARY_le"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 22:
    prediction = "UNARY_eq"
  elif np.argmax(output.logits.cpu().numpy()).flatten().item() == 23:
    prediction = "UNARY_ne"
  else:
    prediction = "UNKOWN"


  return prediction, output.logits.cpu().numpy()

classifModel = loadClassifModel()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 79.4 MB/s eta 0:00:00


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly i

In [ ]:
# c,l = classify("distance between V and C should be greater than gap betwen P and H 8", classifModel)

# Named Entity Recognition

In [ ]:
!git clone https://github.com/CVxTz/ner_playground.git

Cloning into 'ner_playground'...
remote: Enumerating objects: 97, done.
remote: Counting objects: 100% (97/97), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 97 (delta 41), reused 80 (delta 28), pack-reused 0
Receiving objects: 100% (97/97), 29.60 KiB | 14.80 MiB/s, done.
Resolving deltas: 100% (41/41), done.


In [ ]:
!mkdir /content/ner_playground/models

In [ ]:
!gdown 1Ign2-RxfYw-dw1tZmJkoTSWtZnJ6CxLQ
!gdown 19OajoLpAP8PMUYGEU8qzKnEJE-9b1_A1
!gdown 1pDYix4GPjyZZDP29Iq-bncJJzcEOQb3U
!gdown 16ewkqljJbK5stGMr8pMxKb2IfRjp1lWa

Downloading...
From: https://drive.google.com/uc?id=1Ign2-RxfYw-dw1tZmJkoTSWtZnJ6CxLQ
To: /content/ner-bert-binaryData-justparams.ckpt
100% 438M/438M [00:18<00:00, 23.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=19OajoLpAP8PMUYGEU8qzKnEJE-9b1_A1
To: /content/ner-bert-distances-justparams.ckpt
100% 438M/438M [00:16<00:00, 26.4MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pDYix4GPjyZZDP29Iq-bncJJzcEOQb3U
To: /content/ner-bert-distancesUnary-justparams.ckpt
100% 438M/438M [00:18<00:00, 24.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=16ewkqljJbK5stGMr8pMxKb2IfRjp1lWa
To: /content/ner-bert-unaryConstraints-justparams.ckpt
100% 438M/438M [00:26<00:00, 16.5MB/s]


In [ ]:
#!cp /content/drive/MyDrive/BertForNERforCSP/ner-bert-constraints-v2.ckpt /content/ner_playground/models
#!cp /content/drive/MyDrive/BertForNERforCSP/ner-bert-alldiffner0.ckpt /content/ner_playground/models
!cp /content/ner-bert-binaryData-justparams.ckpt /content/ner_playground/models
!cp /content/ner-bert-distances-justparams.ckpt /content/ner_playground/models
!cp /content/ner-bert-distancesUnary-justparams.ckpt /content/ner_playground/models
!cp /content/ner-bert-unaryConstraints-justparams.ckpt /content/ner_playground/models
#!cp /content/drive/MyDrive/BertForNERforCSP/ner-bert-allequalner0.ckpt /content/ner_playground/models

In [ ]:
!pip install -r /content/ner_playground/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 36.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.7/11.7 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.0/584.0 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 68.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.4/78.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 52.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 51.3 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.65.0
    Uninstalling tqdm-4.65.0:
   

In [ ]:
!python /content/ner_playground/scripts/load_save_bert.py

Downloading: 100% 570/570 [00:00<00:00, 3.34MB/s]
Downloading: 100% 420M/420M [00:08<00:00, 50.5MB/s]
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Downloading: 100% 226k/226k [00:00<00:00, 504kB/s]
Do

In [ ]:
#!pip install torch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0 torchtext==0.13.0 torchdata==0.4.0 nervaluate

In [ ]:
#!pip install pytorch_lightning==1.8

In [ ]:
import re
from typing import Dict, List, Optional

In [ ]:
import math

import pytorch_lightning as pl
from torch import nn
from torch.nn import functional as F
from transformers import BertConfig, BertModel


CONFIG_PATH = "/content/ner_playground/bert_model/bert_config.json"
CONFIG = BertConfig.from_json_file(CONFIG_PATH)

import torch
from transformers import get_cosine_schedule_with_warmup


def masked_accuracy(y_true: torch.Tensor, y_pred: torch.Tensor, mask):
    y_true = torch.masked_select(y_true, mask)
    y_pred = torch.masked_select(y_pred, mask)

    acc = (y_true == y_pred).double().mean()

    return acc


class BaseModel(pl.LightningModule):
    def training_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, name="train")

    def validation_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, name="valid")

    def test_step(self, batch, batch_idx):
        return self._step(batch, batch_idx, name="test")

    def _step(self, batch, batch_idx, name="train"):
        x, y = batch

        y_hat = self(x)

        y_hat = y_hat.reshape(-1, y_hat.size(2))
        y = y.view(-1)

        loss = F.cross_entropy(y_hat, y, reduction="mean")

        _, predicted = torch.max(y_hat, 1)

        mask = x != self.pad_idx
        mask = mask.view(-1)

        acc = masked_accuracy(y, predicted, mask)

        self.log(f"{name}_loss", loss)
        self.log(f"{name}_acc", acc)

        return loss

    def configure_optimizers(self):
        opt = torch.optim.AdamW(self.parameters(), lr=self.lr)
        lr_schedulers = {
            "scheduler": get_cosine_schedule_with_warmup(
                optimizer=opt, num_warmup_steps=1000, num_training_steps=7700
            ),
            "name": "learning_rate",
            "interval": "step",
            "frequency": 1,
        }
        return [opt], [lr_schedulers]


class BertNerModel(BaseModel):
    def __init__(
        self,
#        n_classes=len(LABEL_MAPPING),
        n_classes,
        pad_idx,
        lr=1e-4,
        # pad_idx=PAD_IDX,
        dropout=0.2,
        bert_path=None,
        keep_layers=("embeddings", "encoder", "pooler"),
    ):
        super().__init__()

        self.save_hyperparameters()

        self.lr = lr
        self.pad_idx = pad_idx

        self.n_classes = n_classes

        CONFIG.hidden_dropout_prob = dropout

        self.bert = BertModel(config=CONFIG)

        for name, param in self.bert.named_parameters():
            if not any(name.startswith(a) for a in keep_layers):
                param.requires_grad = False

        if bert_path:
            state_dict = torch.load(bert_path)
            self.bert.load_state_dict(state_dict)

        self.do = nn.Dropout(p=dropout)

        self.out_linear = nn.Linear(CONFIG.hidden_size, n_classes)

    def forward(self, x):

        mask = (x != self.pad_idx).int()
        x = self.bert(
            x, attention_mask=mask, encoder_attention_mask=mask
        ).last_hidden_state
        # [batch, Seq_len, CONFIG.hidden_size]

        x = self.do(x)

        out = self.out_linear(x)

        return out


class PositionalEncoding(torch.nn.Module):
    #  https://pytorch.org/tutorials/beginner/transformer_tutorial.html

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = torch.nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model)
        )
        pe = torch.zeros(1, max_len, d_model)
        pe[0:, :, 0::2] = torch.sin(position * div_term)
        pe[0:, :, 1::2] = torch.cos(position * div_term)
        self.register_buffer("pe", pe)

        self.d_model = d_model

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """

        x = x + self.pe[:, : x.size(1)] / math.sqrt(self.d_model)

        return self.dropout(x)


class BaseNerModel(BaseModel):
    def __init__(
        self,
        #n_classes=len(LABEL_MAPPING),
        n_classes,
        pad_idx,
        n_vocab,
        lr=1e-4,
        #pad_idx=PAD_IDX,
        dropout=0.2,
        d_model=9466,
        #n_vocab=N_VOCAB,
    ):
        super().__init__()

        self.save_hyperparameters()

        self.lr = lr
        self.pad_idx = pad_idx
        self.d_model = d_model
        self.dropout = dropout
        self.n_vocab = n_vocab

        self.n_classes = n_classes

        # Text
        self.embeddings = torch.nn.Embedding(self.n_vocab, self.d_model)
        self.pos_encoder = PositionalEncoding(
            d_model=self.d_model, dropout=self.dropout
        )
        encoder_layer = torch.nn.TransformerEncoderLayer(
            d_model=self.d_model, nhead=4, dropout=self.dropout, batch_first=True
        )
        self.encoder = torch.nn.TransformerEncoder(encoder_layer, num_layers=4)

        self.do = nn.Dropout(p=dropout)

        self.out_linear = nn.Linear(self.d_model, n_classes)

    def forward(self, x):

        mask = (x == self.pad_idx).float()

        mask = mask.masked_fill(mask == 1, float("-inf")).masked_fill(
            mask == 0, float(0.0)
        )

        x = self.embeddings(x)
        x = self.pos_encoder(x)
        x = self.encoder(x, src_key_padding_mask=mask)

        x = self.do(x)

        out = self.out_linear(x)

        return out


In [ ]:
from transformers import BertTokenizerFast
from pathlib import Path
from pprint import pprint
import pandas as pd
import torch
from tqdm import tqdm


## ner_binary

In [ ]:
def ner_binary(sentence):
  class Token:
    def __init__(
        self,
        token: str,
        index: int,
        start_index: int,
        end_index: int,
        raw_label: Optional[str] = None,
        bio_label: Optional[str] = None,
    ):
        self.token = token
        self.index = index
        self.start_index = start_index
        self.end_index = end_index
        self.raw_label = raw_label
        self.bio_label = bio_label

    def __repr__(self):
        return (
            f"T: {self.token} / "
            f"I: {self.index} / "
            f"S: {self.start_index} / "
            f"E: {self.end_index} / "
            f"RL: {self.raw_label} / "
            f"CL: {self.clean_label} / "
            f"BIO: {self.bio_label}"
        )

    @property
    def bio_idx(self):
        return LABEL_MAPPING[self.bio_label]

    @property
    def clean_label(self):
        return re.sub(r"\s#\d+$", "", self.raw_label)

    def as_dict(self):
        return {
            "token": self.token,
            "index": self.index,
            "start_index": self.start_index,
            "end_index": self.end_index,
            "raw_label": self.raw_label,
            "bio_label": self.bio_label,
        }

    @classmethod
    def from_dict(cls, as_dict: Dict):
        return cls(**as_dict)


  def tokenize(text: str):
    encoded = TOKENIZER.encode_plus(text, return_offsets_mapping=True)
    ids = encoded["input_ids"]
    offsets = encoded["offset_mapping"]
    tokens = TOKENIZER.convert_ids_to_tokens(ids)

    tokens = [
        Token(token=token, index=index, start_index=offset[0], end_index=offset[1])
        for token, index, offset in zip(tokens, ids, offsets)
    ]

    return tokens


  def most_frequent(list_of_labels):
    return max(set(list_of_labels), key=list_of_labels.count)


  def generate_labeled_tokens(text: str, labels: List[Dict]):
    tokens = tokenize(text=text)

    char_label = ["O"] * len(text)

    for i, span in enumerate(labels):

        label = span["label"]
        start = span["start"]
        end = span["end"]

        char_label[start:end] = [f"{label} #{i}"] * (end - start)

    for i, token in enumerate(tokens):
        if token.start_index != token.end_index:
            token.raw_label = most_frequent(
                char_label[token.start_index : token.end_index]
            )
        else:
            token.raw_label = "O"

    # BIO labels
    for i, token in enumerate(tokens):
        if token.raw_label != "O":
            if i == 0:
                token.bio_label = "B-" + token.clean_label

            else:
                if tokens[i - 1].raw_label == tokens[i].raw_label:
                    token.bio_label = "I-" + token.clean_label
                else:
                    token.bio_label = "B-" + token.clean_label
        else:
            token.bio_label = token.clean_label

    return tokens


  def group_tokens_by_entity(tokens: List[Token]):
    """
    List to List[List[Token]]

    :param tokens:
    :return:
    """
    block_tokens = []
    for i, token in enumerate(tokens):
        if token.bio_label == "O" or token.start_index == token.end_index == 0:
            continue
        elif i == 0:
            block_tokens.append([token])
        elif (
            tokens[i].bio_label.split("-")[0] == "B"
            or tokens[i - 1].bio_label.split("-")[-1]
            != tokens[i].bio_label.split("-")[-1]
        ):
            block_tokens.append([token])
        else:
            block_tokens[-1].append(token)

    return block_tokens


  def decode_labeled_tokens(tokens: List[Token]):
    """
    decode labeled tokens into word indexes

    :param tokens:
    :return:
    """
    block_tokens = group_tokens_by_entity(tokens=tokens)

    labels = []
    for block in block_tokens:
        start = min(token.start_index for token in block)
        end = max(token.end_index for token in block)
        label = block[0].bio_label.split("-")[-1]
        labels.append({"label": label, "start": start, "end": end})

    return labels


  TOKENIZER_PATH = "/content/ner_playground/bert_model"
  TOKENIZER = BertTokenizerFast.from_pretrained(str(TOKENIZER_PATH))
  CLASSES = [
      "DESC",
      "CONST_ELMS_LEFT",
      "CONST_ELMS_RIGHT"
  ]

  LABEL_MAPPING = {
      "O": 0,
  }
  i = 1
  for c in CLASSES:
      LABEL_MAPPING[f"B-{c}"] = i
      LABEL_MAPPING[f"I-{c}"] = i + 1
      i += 2


  INV_LABEL_MAPPING = {v: k for k, v in LABEL_MAPPING.items()}

  CLS = "[CLS]"
  PAD = "[PAD]"
  SEP = "[SEP]"

  PAD_IDX = TOKENIZER.pad_token_id
  CLS_IDX = TOKENIZER.cls_token_id
  SEP_IDX = TOKENIZER.sep_token_id

  N_VOCAB = len(TOKENIZER.get_vocab())

  MAX_LEN = 256


  modeln = "ner-bert-binaryData-justparams.ckpt"
  BASE_PATH = "/content/ner_playground"
  BERT_PATH = BASE_PATH+ "/bert_model/pytorch_model.bin"


  model = BertNerModel(
        lr=5e-5,
        n_classes= len(LABEL_MAPPING),
        pad_idx=PAD_IDX,
    )

  model.eval()

  model_path = BASE_PATH + "/models/"+modeln
  model.load_state_dict(torch.load(model_path)["state_dict"])

  gold_spans = []
  predicted_spans = []

  cpt = 0


  prediction_tokens = generate_labeled_tokens(sentence, labels=[])[:MAX_LEN]

  x = torch.tensor([token.index for token in prediction_tokens], dtype=torch.long)
  x = x.unsqueeze(0)

  with torch.no_grad():

    prediction_score = model(x).squeeze(0)

    _, predicted = torch.max(prediction_score, 1)

    for token, label_index in zip(prediction_tokens, predicted.tolist()):
      token.bio_label = INV_LABEL_MAPPING[label_index]

    predicted_spans.append(decode_labeled_tokens(prediction_tokens))




  return predicted_spans

In [ ]:
# ner_binary("V should be greater than W")

## ner_unary

In [ ]:
def ner_unary(sentence):


  class Token:
    def __init__(
        self,
        token: str,
        index: int,
        start_index: int,
        end_index: int,
        raw_label: Optional[str] = None,
        bio_label: Optional[str] = None,
    ):
        self.token = token
        self.index = index
        self.start_index = start_index
        self.end_index = end_index
        self.raw_label = raw_label
        self.bio_label = bio_label

    def __repr__(self):
        return (
            f"T: {self.token} / "
            f"I: {self.index} / "
            f"S: {self.start_index} / "
            f"E: {self.end_index} / "
            f"RL: {self.raw_label} / "
            f"CL: {self.clean_label} / "
            f"BIO: {self.bio_label}"
        )

    @property
    def bio_idx(self):
        return LABEL_MAPPING[self.bio_label]

    @property
    def clean_label(self):
        return re.sub(r"\s#\d+$", "", self.raw_label)

    def as_dict(self):
        return {
            "token": self.token,
            "index": self.index,
            "start_index": self.start_index,
            "end_index": self.end_index,
            "raw_label": self.raw_label,
            "bio_label": self.bio_label,
        }

    @classmethod
    def from_dict(cls, as_dict: Dict):
        return cls(**as_dict)


  def tokenize(text: str):
    encoded = TOKENIZER.encode_plus(text, return_offsets_mapping=True)
    ids = encoded["input_ids"]
    offsets = encoded["offset_mapping"]
    tokens = TOKENIZER.convert_ids_to_tokens(ids)

    tokens = [
        Token(token=token, index=index, start_index=offset[0], end_index=offset[1])
        for token, index, offset in zip(tokens, ids, offsets)
    ]

    return tokens


  def most_frequent(list_of_labels):
    return max(set(list_of_labels), key=list_of_labels.count)


  def generate_labeled_tokens(text: str, labels: List[Dict]):
    tokens = tokenize(text=text)

    char_label = ["O"] * len(text)

    for i, span in enumerate(labels):

        label = span["label"]
        start = span["start"]
        end = span["end"]

        char_label[start:end] = [f"{label} #{i}"] * (end - start)

    for i, token in enumerate(tokens):
        if token.start_index != token.end_index:
            token.raw_label = most_frequent(
                char_label[token.start_index : token.end_index]
            )
        else:
            token.raw_label = "O"

    # BIO labels
    for i, token in enumerate(tokens):
        if token.raw_label != "O":
            if i == 0:
                token.bio_label = "B-" + token.clean_label

            else:
                if tokens[i - 1].raw_label == tokens[i].raw_label:
                    token.bio_label = "I-" + token.clean_label
                else:
                    token.bio_label = "B-" + token.clean_label
        else:
            token.bio_label = token.clean_label

    return tokens


  def group_tokens_by_entity(tokens: List[Token]):
    """
    List to List[List[Token]]

    :param tokens:
    :return:
    """
    block_tokens = []
    for i, token in enumerate(tokens):
        if token.bio_label == "O" or token.start_index == token.end_index == 0:
            continue
        elif i == 0:
            block_tokens.append([token])
        elif (
            tokens[i].bio_label.split("-")[0] == "B"
            or tokens[i - 1].bio_label.split("-")[-1]
            != tokens[i].bio_label.split("-")[-1]
        ):
            block_tokens.append([token])
        else:
            block_tokens[-1].append(token)

    return block_tokens


  def decode_labeled_tokens(tokens: List[Token]):
    """
    decode labeled tokens into word indexes

    :param tokens:
    :return:
    """
    block_tokens = group_tokens_by_entity(tokens=tokens)

    labels = []
    for block in block_tokens:
        start = min(token.start_index for token in block)
        end = max(token.end_index for token in block)
        label = block[0].bio_label.split("-")[-1]
        labels.append({"label": label, "start": start, "end": end})

    return labels


  TOKENIZER_PATH = "/content/ner_playground/bert_model"
  TOKENIZER = BertTokenizerFast.from_pretrained(str(TOKENIZER_PATH))
  CLASSES = [
      "DESC",
      "VARIABLE",
      "PARAMETER"
  ]



  LABEL_MAPPING = {
      "O": 0,
  }
  i = 1
  for c in CLASSES:
      LABEL_MAPPING[f"B-{c}"] = i
      LABEL_MAPPING[f"I-{c}"] = i + 1
      i += 2


  INV_LABEL_MAPPING = {v: k for k, v in LABEL_MAPPING.items()}

  CLS = "[CLS]"
  PAD = "[PAD]"
  SEP = "[SEP]"

  PAD_IDX = TOKENIZER.pad_token_id
  CLS_IDX = TOKENIZER.cls_token_id
  SEP_IDX = TOKENIZER.sep_token_id

  N_VOCAB = len(TOKENIZER.get_vocab())

  MAX_LEN = 256


  modeln = "ner-bert-unaryConstraints-justparams.ckpt"
  BASE_PATH = "/content/ner_playground"
  BERT_PATH = BASE_PATH+ "/bert_model/pytorch_model.bin"


  model = BertNerModel(
        lr=5e-5,
        n_classes= len(LABEL_MAPPING),
        pad_idx=PAD_IDX,
    )

  model.eval()

  model_path = BASE_PATH + "/models/"+modeln
  model.load_state_dict(torch.load(model_path)["state_dict"])

  gold_spans = []
  predicted_spans = []

  cpt = 0


  prediction_tokens = generate_labeled_tokens(sentence, labels=[])[:MAX_LEN]

  x = torch.tensor([token.index for token in prediction_tokens], dtype=torch.long)
  x = x.unsqueeze(0)

  with torch.no_grad():

    prediction_score = model(x).squeeze(0)

    _, predicted = torch.max(prediction_score, 1)

    for token, label_index in zip(prediction_tokens, predicted.tolist()):
      token.bio_label = INV_LABEL_MAPPING[label_index]

    predicted_spans.append(decode_labeled_tokens(prediction_tokens))




  return predicted_spans

In [ ]:
# ner_unary("X should be greater than 9")

## ner_DistanceBinary

In [ ]:
def ner_DistanceBinary(sentence):

  class Token:
    def __init__(
        self,
        token: str,
        index: int,
        start_index: int,
        end_index: int,
        raw_label: Optional[str] = None,
        bio_label: Optional[str] = None,
    ):
        self.token = token
        self.index = index
        self.start_index = start_index
        self.end_index = end_index
        self.raw_label = raw_label
        self.bio_label = bio_label

    def __repr__(self):
        return (
            f"T: {self.token} / "
            f"I: {self.index} / "
            f"S: {self.start_index} / "
            f"E: {self.end_index} / "
            f"RL: {self.raw_label} / "
            f"CL: {self.clean_label} / "
            f"BIO: {self.bio_label}"
        )

    @property
    def bio_idx(self):
        return LABEL_MAPPING[self.bio_label]

    @property
    def clean_label(self):
        return re.sub(r"\s#\d+$", "", self.raw_label)

    def as_dict(self):
        return {
            "token": self.token,
            "index": self.index,
            "start_index": self.start_index,
            "end_index": self.end_index,
            "raw_label": self.raw_label,
            "bio_label": self.bio_label,
        }

    @classmethod
    def from_dict(cls, as_dict: Dict):
        return cls(**as_dict)


  def tokenize(text: str):
    encoded = TOKENIZER.encode_plus(text, return_offsets_mapping=True)
    ids = encoded["input_ids"]
    offsets = encoded["offset_mapping"]
    tokens = TOKENIZER.convert_ids_to_tokens(ids)

    tokens = [
        Token(token=token, index=index, start_index=offset[0], end_index=offset[1])
        for token, index, offset in zip(tokens, ids, offsets)
    ]

    return tokens


  def most_frequent(list_of_labels):
    return max(set(list_of_labels), key=list_of_labels.count)


  def generate_labeled_tokens(text: str, labels: List[Dict]):
    tokens = tokenize(text=text)

    char_label = ["O"] * len(text)

    for i, span in enumerate(labels):

        label = span["label"]
        start = span["start"]
        end = span["end"]

        char_label[start:end] = [f"{label} #{i}"] * (end - start)

    for i, token in enumerate(tokens):
        if token.start_index != token.end_index:
            token.raw_label = most_frequent(
                char_label[token.start_index : token.end_index]
            )
        else:
            token.raw_label = "O"

    # BIO labels
    for i, token in enumerate(tokens):
        if token.raw_label != "O":
            if i == 0:
                token.bio_label = "B-" + token.clean_label

            else:
                if tokens[i - 1].raw_label == tokens[i].raw_label:
                    token.bio_label = "I-" + token.clean_label
                else:
                    token.bio_label = "B-" + token.clean_label
        else:
            token.bio_label = token.clean_label

    return tokens


  def group_tokens_by_entity(tokens: List[Token]):
    """
    List to List[List[Token]]

    :param tokens:
    :return:
    """
    block_tokens = []
    for i, token in enumerate(tokens):
        if token.bio_label == "O" or token.start_index == token.end_index == 0:
            continue
        elif i == 0:
            block_tokens.append([token])
        elif (
            tokens[i].bio_label.split("-")[0] == "B"
            or tokens[i - 1].bio_label.split("-")[-1]
            != tokens[i].bio_label.split("-")[-1]
        ):
            block_tokens.append([token])
        else:
            block_tokens[-1].append(token)

    return block_tokens


  def decode_labeled_tokens(tokens: List[Token]):
    """
    decode labeled tokens into word indexes

    :param tokens:
    :return:
    """
    block_tokens = group_tokens_by_entity(tokens=tokens)

    labels = []
    for block in block_tokens:
        start = min(token.start_index for token in block)
        end = max(token.end_index for token in block)
        label = block[0].bio_label.split("-")[-1]
        labels.append({"label": label, "start": start, "end": end})

    return labels

  TOKENIZER_PATH = "/content/ner_playground/bert_model"
  TOKENIZER = BertTokenizerFast.from_pretrained(str(TOKENIZER_PATH))
  CLASSES = [
      "DESC",
      "Dist_Right_Elm_1",
      "Dist_Right_Elm_2",
      "Dist_Left_Elm_1",
      "Dist_Left_Elm_2"
  ]





  LABEL_MAPPING = {
      "O": 0,
  }
  i = 1
  for c in CLASSES:
      LABEL_MAPPING[f"B-{c}"] = i
      LABEL_MAPPING[f"I-{c}"] = i + 1
      i += 2


  INV_LABEL_MAPPING = {v: k for k, v in LABEL_MAPPING.items()}

  CLS = "[CLS]"
  PAD = "[PAD]"
  SEP = "[SEP]"

  PAD_IDX = TOKENIZER.pad_token_id
  CLS_IDX = TOKENIZER.cls_token_id
  SEP_IDX = TOKENIZER.sep_token_id

  N_VOCAB = len(TOKENIZER.get_vocab())


  MAX_LEN = 256


  modeln = "ner-bert-distances-justparams.ckpt"
  BASE_PATH = "/content/ner_playground"
  BERT_PATH = BASE_PATH+ "/bert_model/pytorch_model.bin"


  model = BertNerModel(
        lr=5e-5,
        n_classes= len(LABEL_MAPPING),
        pad_idx=PAD_IDX,
    )

  model.eval()

  model_path = BASE_PATH + "/models/"+modeln
  model.load_state_dict(torch.load(model_path)["state_dict"])

  gold_spans = []
  predicted_spans = []

  cpt = 0


  prediction_tokens = generate_labeled_tokens(sentence, labels=[])[:MAX_LEN]

  x = torch.tensor([token.index for token in prediction_tokens], dtype=torch.long)
  x = x.unsqueeze(0)

  with torch.no_grad():

    prediction_score = model(x).squeeze(0)

    _, predicted = torch.max(prediction_score, 1)

    for token, label_index in zip(prediction_tokens, predicted.tolist()):
      token.bio_label = INV_LABEL_MAPPING[label_index]

    predicted_spans.append(decode_labeled_tokens(prediction_tokens))




  return predicted_spans

In [ ]:
# ner_DistanceBinary("d1 and D2 should be greater than D3 and D4")

## ner_DistanceUnary

In [ ]:
def ner_DistanceUnary(sentence):

  class Token:
    def __init__(
        self,
        token: str,
        index: int,
        start_index: int,
        end_index: int,
        raw_label: Optional[str] = None,
        bio_label: Optional[str] = None,
    ):
        self.token = token
        self.index = index
        self.start_index = start_index
        self.end_index = end_index
        self.raw_label = raw_label
        self.bio_label = bio_label

    def __repr__(self):
        return (
            f"T: {self.token} / "
            f"I: {self.index} / "
            f"S: {self.start_index} / "
            f"E: {self.end_index} / "
            f"RL: {self.raw_label} / "
            f"CL: {self.clean_label} / "
            f"BIO: {self.bio_label}"
        )

    @property
    def bio_idx(self):
        return LABEL_MAPPING[self.bio_label]

    @property
    def clean_label(self):
        return re.sub(r"\s#\d+$", "", self.raw_label)

    def as_dict(self):
        return {
            "token": self.token,
            "index": self.index,
            "start_index": self.start_index,
            "end_index": self.end_index,
            "raw_label": self.raw_label,
            "bio_label": self.bio_label,
        }

    @classmethod
    def from_dict(cls, as_dict: Dict):
        return cls(**as_dict)


  def tokenize(text: str):
    encoded = TOKENIZER.encode_plus(text, return_offsets_mapping=True)
    ids = encoded["input_ids"]
    offsets = encoded["offset_mapping"]
    tokens = TOKENIZER.convert_ids_to_tokens(ids)

    tokens = [
        Token(token=token, index=index, start_index=offset[0], end_index=offset[1])
        for token, index, offset in zip(tokens, ids, offsets)
    ]

    return tokens


  def most_frequent(list_of_labels):
    return max(set(list_of_labels), key=list_of_labels.count)


  def generate_labeled_tokens(text: str, labels: List[Dict]):
    tokens = tokenize(text=text)

    char_label = ["O"] * len(text)

    for i, span in enumerate(labels):

        label = span["label"]
        start = span["start"]
        end = span["end"]

        char_label[start:end] = [f"{label} #{i}"] * (end - start)

    for i, token in enumerate(tokens):
        if token.start_index != token.end_index:
            token.raw_label = most_frequent(
                char_label[token.start_index : token.end_index]
            )
        else:
            token.raw_label = "O"

    # BIO labels
    for i, token in enumerate(tokens):
        if token.raw_label != "O":
            if i == 0:
                token.bio_label = "B-" + token.clean_label

            else:
                if tokens[i - 1].raw_label == tokens[i].raw_label:
                    token.bio_label = "I-" + token.clean_label
                else:
                    token.bio_label = "B-" + token.clean_label
        else:
            token.bio_label = token.clean_label

    return tokens


  def group_tokens_by_entity(tokens: List[Token]):
    """
    List to List[List[Token]]

    :param tokens:
    :return:
    """
    block_tokens = []
    for i, token in enumerate(tokens):
        if token.bio_label == "O" or token.start_index == token.end_index == 0:
            continue
        elif i == 0:
            block_tokens.append([token])
        elif (
            tokens[i].bio_label.split("-")[0] == "B"
            or tokens[i - 1].bio_label.split("-")[-1]
            != tokens[i].bio_label.split("-")[-1]
        ):
            block_tokens.append([token])
        else:
            block_tokens[-1].append(token)

    return block_tokens


  def decode_labeled_tokens(tokens: List[Token]):
    """
    decode labeled tokens into word indexes

    :param tokens:
    :return:
    """
    block_tokens = group_tokens_by_entity(tokens=tokens)

    labels = []
    for block in block_tokens:
        start = min(token.start_index for token in block)
        end = max(token.end_index for token in block)
        label = block[0].bio_label.split("-")[-1]
        labels.append({"label": label, "start": start, "end": end})

    return labels

  TOKENIZER_PATH = "/content/ner_playground/bert_model"
  TOKENIZER = BertTokenizerFast.from_pretrained(str(TOKENIZER_PATH))
  CLASSES = [
      "DESC",
      "Dist_Left_Elm_1",
      "Dist_Left_Elm_2",
      "Parameter"
  ]





  LABEL_MAPPING = {
      "O": 0,
  }
  i = 1
  for c in CLASSES:
      LABEL_MAPPING[f"B-{c}"] = i
      LABEL_MAPPING[f"I-{c}"] = i + 1
      i += 2


  INV_LABEL_MAPPING = {v: k for k, v in LABEL_MAPPING.items()}

  CLS = "[CLS]"
  PAD = "[PAD]"
  SEP = "[SEP]"

  PAD_IDX = TOKENIZER.pad_token_id
  CLS_IDX = TOKENIZER.cls_token_id
  SEP_IDX = TOKENIZER.sep_token_id

  N_VOCAB = len(TOKENIZER.get_vocab())




  MAX_LEN = 256


  modeln = "ner-bert-distancesUnary-justparams.ckpt"
  BASE_PATH = "/content/ner_playground"
  BERT_PATH = BASE_PATH+ "/bert_model/pytorch_model.bin"


  model = BertNerModel(
        lr=5e-5,
        n_classes= len(LABEL_MAPPING),
        pad_idx=PAD_IDX,
    )

  model.eval()

  model_path = BASE_PATH + "/models/"+modeln
  model.load_state_dict(torch.load(model_path)["state_dict"])

  gold_spans = []
  predicted_spans = []

  cpt = 0


  prediction_tokens = generate_labeled_tokens(sentence, labels=[])[:MAX_LEN]

  x = torch.tensor([token.index for token in prediction_tokens], dtype=torch.long)
  x = x.unsqueeze(0)

  with torch.no_grad():

    prediction_score = model(x).squeeze(0)

    _, predicted = torch.max(prediction_score, 1)

    for token, label_index in zip(prediction_tokens, predicted.tolist()):
      token.bio_label = INV_LABEL_MAPPING[label_index]

    predicted_spans.append(decode_labeled_tokens(prediction_tokens))




  return predicted_spans

In [ ]:
# ner_DistanceUnary("X1 and X2 must be less than 1")

# Parsers

In [ ]:
labels = {
    "0" : "DESC",
    "1":"CONST_ELMS_LEFT",
    "2":"CONST_ELMS_LEFT_TYPE",
    "3":"CONST_ELMS_RIGHT",
    "4":"CONST_ELMS_RIGHT_TYPE",
    "5":"gt",
    "6":"ge",
    "7":"lt",
    "8":"le",
    "9":"eq",
    "10":"ne",
    "11":"alldiff",
    "12":"allequ"
}

# isCommutative for distances means | a - b | is the same as | b - a | because we have absolute value
operators_mappings = {
    "gt" : {
        "operator": operator.gt,
        "arity": 2,
        "isCommutative":False,
        "parameters" : None
    },
    "ge" : {
        "operator": operator.ge,
        "arity": 2,
        "isCommutative":False,
        "parameters" : None
    },
    "lt" : {
        "operator": operator.lt,
        "arity": 2,
        "isCommutative":False,
        "parameters" : None
    },
    "le" : {
        "operator": operator.le,
        "arity": 2,
        "isCommutative":False,
        "parameters" : None
    },
    "ne" : {
        "operator": operator.ne,
        "arity": 2,
        "isCommutative":True,
        "parameters" : None
    },
    "eq" : {
        "operator": operator.eq,
        "arity": 2,
        "isCommutative":True,
        "parameters" : None
    },
    "gtConst":
     {
        "operator": gtConst,
        "arity": 1,
        "isCommutative":False,
        "parameters" : []
    },
    "geConst":
     {
        "operator": geConst,
        "arity": 1,
        "isCommutative":False,
        "parameters" : []
    },
    "ltConst":
     {
        "operator": ltConst,
        "arity": 1,
        "isCommutative":False,
        "parameters" : []
    },
    "leConst":
     {
        "operator": leConst,
        "arity": 1,
        "isCommutative":False,
        "parameters" : []
    },
    "neConst":
     {
        "operator": neConst,
        "arity": 1,
        "isCommutative":True,
        "parameters" : []
    },
    "eqConst":
     {
        "operator": eqConst,
        "arity": 1,
        "isCommutative":True,
        "parameters" : []
    },
    "DistgtValue":{
        "operator": DistgtValue,
        "arity": 2,
        "isCommutative":True,
        "parameters" : []
    },
    "DistgeValue":{
        "operator": DistgeValue,
        "arity": 2,
        "isCommutative":True,
        "parameters" : []
    },
    "DistltValue":{
        "operator": DistltValue,
        "arity": 2,
        "isCommutative":True,
        "parameters" : []
    },
    "DistleValue":{
        "operator": DistleValue,
        "arity": 2,
        "isCommutative":True,
        "parameters" : []
    },
    "DisteqValue":{
        "operator": DisteqValue,
        "arity": 2,
        "isCommutative":True,
        "parameters" : []
    },
    "DistneValue":{
        "operator": DistneValue,
        "arity": 2,
        "isCommutative":True,
        "parameters" : []
    },
    "DisteqDist":{
        "operator": eqDist,
        "arity": 4,
        "isCommutative":True,
        "parameters" : None
    },
    "DisteqDist3":{
        "operator": eqDist3,
        "arity": 3,
        "isCommutative":False,
        "parameters" : None
    },
    "DistneDist":{
        "operator": neDist,
        "arity": 4,
        "isCommutative":True,
        "parameters" : None
    },
    "DistneDist3":{
        "operator": neDist3,
        "arity": 3,
        "isCommutative":False,
        "parameters" : None
    },
    "DistgtDist":{
        "operator": gtDist,
        "arity": 4,
        "isCommutative":False,
        "parameters" : None
    },
    "DistgeDist3":{
        "operator": geDist3,
        "arity": 3,
        "isCommutative":False,
        "parameters" : None
    },
    "DistltDist":{
        "operator": ltDist,
        "arity": 4,
        "isCommutative":False,
        "parameters" : None
    },
    "DistleDist3":{
        "operator": leDist3,
        "arity": 3,
        "isCommutative":False,
        "parameters" : None
    }


    # "distanceUnary":
    # "distanceBinary":
    # "distanceTurnary":
}


constTypes = list(operators_mappings.keys())

## ParserUnary

In [ ]:
def parseUnary(text,predicted_spans):

  constraints = []

  i = 0
  # get indexes of constraint types (e.g. gt, lt ...)
  # these indexes will be used to begin the search
  # for the other attributes const_elms_left and const_elms_right

  constraint = {} # begin constructing the constraint
  while(i<len(predicted_spans[0])):
    if(predicted_spans[0][i]["label"] == "VARIABLE"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    elif(predicted_spans[0][i]["label"] == "PARAMETER"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    i+=1

  return constraint

In [ ]:
# t = "W should be greater than 4"
# parseUnary(t,ner_unary(t))

In [ ]:
import re

def formulateUnary(text, c, const_type):


  if( "VARIABLE" in c.keys() ):
    variable = c["VARIABLE"]
  else:
    return False, ([] , None, None), None


  if( "PARAMETER" in c.keys() ):
    try:
      parameter = [int(c["PARAMETER"])]
    except:
      return False, ([] , None, None), None
  else:
    return False, ([] , None, None), None

  scope_ids = []
  for k in problem_data.keys():
      if problem_data[k]["name"] == variable:
        scope_ids.append(k)

  if len(scope_ids) == operators_mappings[const_type]["arity"]:
    print(scope_ids ,
              operators_mappings[const_type]["operator"],
              operators_mappings[const_type]["arity"],
              operators_mappings[const_type]["isCommutative"],
              parameter
            )
    return True, Constraint( scope_ids ,
              operators_mappings[const_type]["operator"],
              operators_mappings[const_type]["arity"],
              operators_mappings[const_type]["isCommutative"],
              parameters = parameter
            ), scope_ids
  else:
    print("We couldn't find the scope")
    print("Here is the constraint: ", operators_mappings[const_type]["operator"])
    return False, ([] , variable, None), scope_ids

In [ ]:

# vars_ids =   [   1 ,   2   ,  3    ,  4     ,   5  ,  6  ,    7   ,   8 ,    9  ,   10  ,   11   ,  12 ,  13  ,  14          ,   15  ,  16     , 17       ,18        ,19          ,20        ,21  , 22, 23,24 ,25]
# vars_names = ["W","green","ivory","yellow","blue","dog","snails","fox","horse","zebra","coffee","tea","milk","orange juice","water","English","Sparniad","Ukranian","Norweigian","Japanese","ol","k","c","L","P"]
# vars_types = ["color","color","color","color","color","pet","pet","pet","pet","pet","drink","drink","drink","drink","drink","country","country","country","country","country","brand","brand","brand","brand","brand"]
# vars_domains = [(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5)]
# types = ["color","pet","drink", "country","brand"]


# problem_data = {
#   id: {
#       "domain": vars_domains[id-1],
#        "type": vars_types[id-1],
#        "name": vars_names[id-1]
#   } for id in vars_ids
# }

# t = "W should be greater than 4"
# formulateUnary(t,parseUnary(t,ner_unary(t)),"gtConst")


## ParserBinary

In [ ]:
def parseBinary(text,predicted_spans):


  i = 0

  constraint = {} # begin constructing the constraint

  # get indexes of constraint types (e.g. gt, lt ...)
  # these indexes will be used to begin the search
  # for the other attributes const_elms_left and const_elms_right
  while(i<len(predicted_spans[0])):
    if(predicted_spans[0][i]['label'] == "CONST_ELMS_LEFT"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    elif(predicted_spans[0][i]['label'] == "CONST_ELMS_RIGHT"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    i+=1

  return constraint

In [ ]:
# t = "W should be greater than V"
# parseBinary(t,ner_binary(t))

In [ ]:
import re

def formulateBinary(text, c, const_type):

  if( "CONST_ELMS_LEFT" in c.keys() ):
    elm_left = c["CONST_ELMS_LEFT"]
  else:
    elm_left = None

  if( "CONST_ELMS_RIGHT" in c.keys() ):
    elm_right = c["CONST_ELMS_RIGHT"]
  else:
    elm_right = None


  #print("CONST_TYPEEEE", const_type)
  if(elm_left == [] and elm_right == []):
    scope = []

  elif(elm_left==[]):
    scope = [elm_right]

  elif(elm_right==[]):
    scope = [elm_left]

  else:
    scope= [elm_left, elm_right]



  scope_ids = []

  for s in scope:
    for k in problem_data.keys():
      if problem_data[k]["name"] == s:
        scope_ids.append(k)

  if len(scope_ids) == operators_mappings[const_type]["arity"]:
    print(scope_ids ,
              operators_mappings[const_type]["operator"],
              operators_mappings[const_type]["arity"],
              operators_mappings[const_type]["isCommutative"])
    return True, Constraint( scope_ids ,
              operators_mappings[const_type]["operator"],
              operators_mappings[const_type]["arity"],
              operators_mappings[const_type]["isCommutative"]
            ), scope_ids
  else:
    print("We couldn't find the scope")
    print("Here is the constraint: ", operators_mappings[const_type]["operator"])
    return False, ([] , scope, None), scope

In [ ]:

# vars_ids =   [   1 ,   2   ,  3    ,  4     ,   5  ,  6  ,    7   ,   8 ,    9  ,   10  ,   11   ,  12 ,  13  ,  14          ,   15  ,  16     , 17       ,18        ,19          ,20        ,21  , 22, 23,24 ,25]
# vars_names = ["A","B","ivory","yellow","blue","dog","snails","fox","horse","zebra","coffee","tea","milk","orange juice","water","English","Sparniad","Ukranian","Norweigian","Japanese","ol","k","c","L","P"]
# vars_types = ["color","color","color","color","color","pet","pet","pet","pet","pet","drink","drink","drink","drink","drink","country","country","country","country","country","brand","brand","brand","brand","brand"]
# vars_domains = [(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5)]
# types = ["color","pet","drink", "country","brand"]


# problem_data = {
#   id: {
#       "domain": vars_domains[id-1],
#        "type": vars_types[id-1],
#        "name": vars_names[id-1]
#   } for id in vars_ids
# }


# t= "the value of A should be less than the value of B"
# ner_binary(t), parseBinary(t, ner_binary(t)) ,formulateBinary(t,parseBinary(t, ner_binary(t)), "gt")


## ParserDistanceUnary

In [ ]:
# t = " distance between A and B should be less than 2"
# ner_DistanceUnary(t)

In [ ]:
def parseDistanceUnary(text,predicted_spans):

  i=0
  constraint = {} # begin constructing the constraint
  while(i<len(predicted_spans[0])):
    if(predicted_spans[0][i]["label"] == "Dist_Left_Elm_1"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    elif(predicted_spans[0][i]["label"] == "Dist_Left_Elm_2"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    elif(predicted_spans[0][i]["label"] == "Parameter"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    i+=1



  return constraint

In [ ]:
# t = " distance between A and B should be less than 2"
# parseDistanceUnary(t,ner_DistanceUnary(t))

In [ ]:
import re

def formulateDistanceUnary(text, c, const_type):

# "DistneValue",
# "Dist_Left_Elm_1",
# "Dist_Left_Elm_2",
# "Parameter"


  # Dist_Left_Elm_1
  if( "Dist_Left_Elm_1" in c.keys() ):
    dist_left_elm1 = c["Dist_Left_Elm_1"]
  else:
    dist_left_elm1 = []

  # Dist_Left_Elm_2
  if( "Dist_Left_Elm_2" in c.keys() ):
    dist_left_elm2 = c["Dist_Left_Elm_2"]
  else:
    dist_left_elm2 = []

  # Parameter

  if("Parameter" in c.keys() ):
    try:
      parameter = [int(c["Parameter"])]
    except:
      return False, ([] , None, None), None
  else:
    return False, ([] , None, None), None



  if(dist_left_elm1 == [] or dist_left_elm2 == []):
    scope = []
  else:
    scope = [dist_left_elm1,dist_left_elm2]


  scope_ids = []

  for s in scope:
    for k in problem_data.keys():
      if problem_data[k]["name"] == s:
        scope_ids.append(k)

  if len(scope_ids) == operators_mappings[const_type]["arity"] and parameter!=[]:
    print(   scope_ids,
              operators_mappings[const_type]["operator"],
              operators_mappings[const_type]["arity"],
              operators_mappings[const_type]["isCommutative"],
              parameter
          )
    return True, Constraint( scope_ids ,
                        operators_mappings[const_type]["operator"],
                        operators_mappings[const_type]["arity"],
                        operators_mappings[const_type]["isCommutative"],
                        parameter
            ), scope_ids
  else:
    print("We couldn't find the scope or the parameter")
    print("Here is the constraint: ", operators_mappings[const_type]["operator"])
    return False, ([] , scope, None), scope

In [ ]:
# formulateDistanceUnary("A and B should be less than 4",parseDistanceUnary("A and B should be less than 4",ner_DistanceUnary("A and B should be less than 4")), "DistltValue")

## ParserDistanceBinary

In [ ]:
# t = "distance between A 2 and F should be equal to distance between Zebra and bsvd"
# ner_DistanceBinary(t)

In [ ]:
def parseDistanceBinary(text,predicted_spans):
  i=0
  constraint = {} # begin constructing the constraint
  while(i<len(predicted_spans[0])):
    if(predicted_spans[0][i]["label"] == "Dist_Left_Elm_1"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    elif(predicted_spans[0][i]["label"] == "Dist_Left_Elm_2"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    elif(predicted_spans[0][i]["label"] == "Dist_Right_Elm_1"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    elif(predicted_spans[0][i]["label"] == "Dist_Right_Elm_2"):
      constraint[predicted_spans[0][i]['label']] = text[predicted_spans[0][i]["start"] : predicted_spans[0][i]["end"]]
    i+=1

  return constraint

In [ ]:
# t = "distance between A 2 and F should be equal to distance between Zebra and bsvd"
# parseDistanceBinary(t,ner_DistanceBinary(t))

In [ ]:
import re

def formulateDistanceBinary(text, c, const_type):


  # Dist_Left_Elm_1

  if( "Dist_Left_Elm_1" in c.keys()):
    dist_left_elm1 = c["Dist_Left_Elm_1"]
  else:
    dist_left_elm1 = []


  if( "Dist_Left_Elm_2" in c.keys()):
    dist_left_elm2 = c["Dist_Left_Elm_2"]
  else:
    dist_left_elm2 = []


  if( "Dist_Right_Elm_1" in c.keys()):
    dist_right_elm1 = c["Dist_Right_Elm_1"]
  else:
    dist_right_elm1 = []


  if( "Dist_Right_Elm_2" in c.keys()):
    dist_right_elm2 = c["Dist_Right_Elm_2"]
  else:
    dist_right_elm2 = []

# TODO: check if an id is repeated
  if(
      dist_right_elm2 == []
      or
      dist_right_elm1 == []
      or
      dist_left_elm1 == []
      or
      dist_left_elm2 == []
      ):
    scope = []
  else:
    scope = [dist_left_elm1,dist_left_elm2,dist_right_elm1,dist_right_elm2]




  scope_ids = []
  for s in scope:
    for k in problem_data.keys():
      if problem_data[k]["name"] == s:
        scope_ids.append(k)


  if len(scope_ids) == operators_mappings[const_type]["arity"]:
    print(scope_ids ,
              operators_mappings[const_type]["operator"],
              operators_mappings[const_type]["arity"],
              operators_mappings[const_type]["isCommutative"])
    return True ,Constraint( scope_ids ,
              operators_mappings[const_type]["operator"],
              operators_mappings[const_type]["arity"],
              operators_mappings[const_type]["isCommutative"]
            ), scope_ids
  else:
    print("We couldn't find the scope")
    print("Here is the constraint: ", operators_mappings[const_type]["operator"])
    return False, ([] , scope, None), scope

In [ ]:
# vars_ids =   [   1 ,   2   ,  3    ,  4     ,   5  ,  6  ,    7   ,   8 ,    9  ,   10  ,   11   ,  12 ,  13  ,  14          ,   15  ,  16     , 17       ,18        ,19          ,20        ,21  , 22, 23,24 ,25]
# vars_names = ["X","Y","C","B","blue","dog","snails","fox","horse","zebra","coffee","tea","milk","orange juice","water","English","Sparniad","Ukranian","Norweigian","Japanese","ol","k","c","L","P"]
# vars_types = ["color","color","color","color","color","pet","pet","pet","pet","pet","drink","drink","drink","drink","drink","country","country","country","country","country","brand","brand","brand","brand","brand"]
# vars_domains = [(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5),(1,5)]
# types = ["color","pet","drink", "country","brand"]


# problem_data = {
#   id: {
#       "domain": vars_domains[id-1],
#        "type": vars_types[id-1],
#        "name": vars_names[id-1]
#   } for id in vars_ids
# }


# t = "the distance from X to Y should be different than the distance between C and B"


# formulateDistanceBinary(t,parseDistanceBinary(t, ner_DistanceBinary(t)),"DistneDist")

# pyQuAcqNLP

## QuAcq

In [ ]:
def ConstraintinListOfConstraints(a,listOfConsts):

  for l in listOfConsts:
    if(a.isEqualTo(l)):
      return True

  return False


def existAlready2(res,c):

    if(ConstraintinListOfConstraints(c, res)):
        return True
    res.append(c)
    return False

def removeDuplicates(listOfConstraints):
  res = []
  nonDup = []
  for c in listOfConstraints:
    if(existAlready2(res,c)):
      continue
    else:
      nonDup.append(c)

  # print("\nBefore remove inside function: ", len(listOfConstraints))
  # print("\nAfter remove inside function: ", len(nonDup))
  return nonDup

def RegularFindC(e,Y,L,B):
  global cpt
  global newsize
  global oldsizeexamples
  global newsizeexamples
  global nbExamplesc
  global nbExamplesa
  global Exampletimes
  global byQuAcq
  global newsizenlpqueries1
  global newsizenlpqueries2
  global newsizenlpqueries3
  global nbConfirmationQueriesC
  global nbConfirmationQueriesA
  global oldsizenlpqueries2
  global byNLP


  delta = []

  delta = B.ConstraintsIsExactlyY(Y)

  tmpp = delta.networkOfConstraintsThatRejectE(e)

  # tmpp.constraints = removeDuplicates(tmpp.constraints)

  delta = joinNetworks( delta , tmpp)

  global findEPrimeCall
  while(True):

    print("\nSTARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall: ",findEPrimeCall,"---------------------\n\n\n")
    print("\n\nDelta before findEPrime: \n")
    for c in delta.constraints:
      print(c.rel, c.scope_ids)


    print("\n\nL[Y] before findEPrime: \n")
    for c in L.ConstraintsIncludedInY(Y).constraints:
      print(c.rel, c.scope_ids)


    stg = time.time()
    ep, ll, d = findEPrime(L,Y,delta)
    #register example sizes and times to generate them
    Exampletimes.append(time.time()-stg)

    #calculate number of examples generated until convergence and until finding an equivalent network
    nbExamplesc+=1
    if(newsizeexamples > oldsizeexamples):
      oldsizeexamples = newsizeexamples
      if(nbExamplesc > nbExamplesa):
        nbExamplesa = nbExamplesc


    print("\n Example: ", ep)

    print("\n\nDelta returned from findEPrime: \n")
    for c in delta.constraints:
      print(c.rel, c.scope_ids)


    print("\n\nL[Y] returned from findEPrime: \n")
    for c in L.ConstraintsIncludedInY(Y).constraints:
      print(c.rel, c.scope_ids)

    print("\nENDENDENDENDENDENDENDENDENDENDEND findEPrimeCall: ",findEPrimeCall,"---------------------\n\n\n")
    findEPrimeCall+=1
    if ep == []:
      print("\n A new constraint was added to the being learned network !!!:\n")
      elementary = [c for c in delta.constraints if isinstance(c,Constraint)]
      Conjs = [c for c in delta.constraints if isinstance(c,Conjunction)]

      if(len(elementary) != 0):
        c = elementary[0]
      else:
        min = 100000
        c = None
        for cc in Conjs:
          if(len(cc.elementaryConstraints)<min):
            min = len(cc.elementaryConstraints)
            c = cc
      #c = delta.constraints[0]


      L = L.addConstraint(c)
      byQuAcq+=1
      newsize+=1
      newsizeexamples+=1
      newsizenlpqueries1+=1
      newsizenlpqueries2+=1
      newsizenlpqueries3+=1

      print("\n Number of constraints so far: \n", len(L.constraints))
      B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(Y).constraints)
      print("\n FindC : Remaining constraints in the Basis: \n", len(B.constraints))
      # print(input())
      #break
      return Y
    else:
      if(ask(ep,Target)[0]):
        delta = delta.removeConstraintsThatRejectE(ep)

        B = B.removeConstraintsThatRejectE(ep)

        print("\n FindC : B reduced in size, new size: ",len(B.constraints))
      else:

        S = FindScope(ep,[],Y,B)
        if set(S).issubset(set(Y)) and len(S)!=len(Y) : #and S!=[]:
          RegularFindC(ep,S,L,B)
        else:
          tmp1 = delta
          tmp11 = tmp1.networkOfConstraintsThatRejectE(ep)
          delta = joinNetworks(tmp1, tmp11)

          print("\nAfter join findcloop: ")
          print(len(delta.constraints))

In [ ]:
# main

def RegularQuAcq2(B,L):
  global Target
  global problem_data
  global redundants
  global nbExamplesc
  global nbExamplesa
  global oldsizeexamples
  global newsizeexamples
  global Exampletimes



  print("In QuAcq2: len of constraints in B is: ", len(B.constraints))
  cpt = 0
  while(True):

    stg = time.time()
    e = generateExample(list(problem_data.keys()),L, B)
    #register example times to generate them
    Exampletimes.append(time.time()-stg)

    #calculate number of examples generated until convergence and until finding an equivalent network
    nbExamplesc+=1
    if(newsizeexamples > oldsizeexamples):
      oldsizeexamples = newsizeexamples
      if(nbExamplesc > nbExamplesa):
        nbExamplesa = nbExamplesc


    if(e==[]):
      return L,None # convergence

    if(ask(e, Target)[0]):

      B = B.removeConstraintsThatRejectE(e)
      print("\nQuacq B reduced in size, new size: ",len(B.constraints))
    else:
      scope_ids = RegularFindC(e, FindScope(e , [] , list(problem_data.keys()), B ), L, B)
      return L,scope_ids


## getConstraint

In [ ]:
def getConstraint(text):
  global Target
  global problem_data
  global classifModel
  global ClassifyTimes
  global NERTimes


  tclass = time.time()
  pred, logits = classify(text, classifModel)
  ClassifyTimes.append(time.time()-tclass)

  t = pred.split("_")

  if t[0] == "UNARY":
    if t[1] == "gt":
      const_type = "gtConst"
    elif t[1] == "ge":
      const_type = "geConst"
    elif t[1] == "lt":
      const_type = "ltConst"
    elif t[1] == "le":
      const_type = "leConst"
    elif t[1] == "eq":
      const_type = "eqConst"
    elif t[1] == "ne":
      const_type = "neConst"

    tner = time.time()
    resNER = ner_unary(text)
    NERTimes.append(time.time() - tner)

    return formulateUnary(text,parseUnary(text,resNER),const_type)

  elif t[0] == "UNARYDIST":


    if t[1] == "gt":
      const_type = "DistgtValue"
    elif t[1] == "ge":
      const_type = "DistgeValue"
    elif t[1] == "lt":
      const_type = "DistltValue"
    elif t[1] == "le":
      const_type = "DistleValue"
    elif t[1] == "eq":
      const_type = "DisteqValue"
    elif t[1] == "ne":
      const_type = "DistneValue"

    tner = time.time()
    resNER = ner_DistanceUnary(text)
    NERTimes.append(time.time() - tner)

    return formulateDistanceUnary(text,parseDistanceUnary(text,resNER),const_type)

  elif t[0] == "BINARYDIST":

    if t[1] == "gt":
      const_type = "DistgtDist"
    elif t[1] == "ge":
      const_type = "DistgeDist"
    elif t[1] == "lt":
      const_type = "DistltDist"
    elif t[1] == "le":
      const_type = "DistleDist"
    elif t[1] == "eq":
      const_type = "DisteqDist"
    elif t[1] == "ne":
      const_type = "DistneDist"


    tner = time.time()
    resNER = ner_DistanceBinary(text)
    NERTimes.append(time.time() - tner)


    return formulateDistanceBinary(text,parseDistanceBinary(text,resNER),const_type)

  elif t[0] == "BINARY":
    const_type = t[1]

    tner = time.time()
    resNER = ner_binary(text)
    NERTimes.append(time.time() - tner)

    return formulateBinary(text,parseBinary(text,resNER),const_type)



## conversation

In [ ]:
def conversation(e, cons):

  global oldsizenlpqueries1
  global newsizenlpqueries1

  global oldsizenlpqueries2
  global newsizenlpqueries2

  global oldsizenlpqueries3
  global newsizenlpqueries3

  global nlpConstraints
  global Examplesizes

  global nbWhatIsWrongQueriesA
  global nbWhatIsWrongQueriesC

  global nbConfirmationQueriesA
  global nbConfirmationQueriesC

  global nbReformulationQueriesA
  global nbReformulationQueriesC

  Examplesizes.append(len(e))


  nbWhatIsWrongQueriesC += 1


  if(newsizenlpqueries1 > oldsizenlpqueries1):
    oldsizenlpqueries1 = newsizenlpqueries1
    if(nbWhatIsWrongQueriesC > nbWhatIsWrongQueriesA):
      nbWhatIsWrongQueriesA = nbWhatIsWrongQueriesC

  print("\n\n Can you tell what is wrong with this example: ")
  #print(ex)
  # mat = []

  # cpt = 1
  # for i in range(3):
  #   mat.append([])
  #   for j in range(4):
  #     mat[i].append(cpt)
  #     cpt+=1


  # for ee in e:
  #   for i in range(3):
  #     for j in range(4):
  #       if(ee[0] == mat[i][j]):
  #         mat[i][j] = ee

  # for i in range(len(mat)):
  #   print(mat[i])
  #   print()
  print(e)

  print("\n\n")

  #problem = input()

  for c in coords:
    print("Here !!!!")
    print(c["indice"],list(c["scope"]), c["rel"],"\n")
    print(cons.scope_ids, cons.rel, cons.parameters,"\n")
    print("----------\n")
    print(list(c["scope"]) == cons.scope_ids and c["rel"].__name__ == cons.rel.__name__)
    #print(input())
    if list(c["scope"]) == cons.scope_ids and c["rel"].__name__ == cons.rel.__name__:
      indice = c["indice"]
  print(nlpConstraints[indice])
  problem = nlpConstraints[indice]
  #nlpConstraints = nlpConstraints[:indice] + nlpConstraints[indice+1:]

  if(problem == "Nothing wrong"): # train a model to detect similar expressions
    return "Converged", []
  elif(problem == "Try to find it yourself"): # train a model to detect similar expressions
    return "FindC", []


  res, c , scope_ids = getConstraint(problem)
  #print("\n\nConstraints: ", constraints)
  #print("\n\nL's Constraints: ", L.constraints)

  while(True):
    if res:
      # add confirmation query
      nbConfirmationQueriesC += 1


      if(newsizenlpqueries2 > oldsizenlpqueries2):
        oldsizenlpqueries2 = newsizenlpqueries2
        if(nbConfirmationQueriesC > nbConfirmationQueriesA):
          nbConfirmationQueriesA = nbConfirmationQueriesC

      print("Is this what you meant ?:\n", c.rel,c.scope_ids,c.isCommutative,c.parameters,c.arity)
      print("cons: \n", cons.rel,cons.scope_ids,cons.isCommutative,cons.parameters,cons.arity)
      print("test: \n", c.rel == cons.rel and c.scope_ids == cons.scope_ids and c.parameters == cons.parameters and c.isCommutative == cons.isCommutative and c.arity == cons.arity)
      print("one by one: \n")
      print("c.rel == cons.rel: ",c.rel == cons.rel)
      print("c.scope_ids == cons.scope_ids ",c.scope_ids == cons.scope_ids)
      print("c.parameters == cons.parameters: ",c.parameters == cons.parameters)
      print("c.isCommutative == cons.isCommutative: ",c.isCommutative == cons.isCommutative)
      print("c.arity == cons.arity: ",c.arity == cons.arity)

      if c.rel == cons.rel and c.scope_ids == cons.scope_ids and c.parameters == cons.parameters and c.isCommutative == cons.isCommutative and c.arity == cons.arity:
        return c, scope_ids
      else:
        print("Sorry for this confusion, would you reformulate it ? or let me try to find it")
        nbReformulationQueriesC += 1
        if(newsizenlpqueries3 > oldsizenlpqueries3):
          oldsizenlpqueries3 = newsizenlpqueries3
          if(nbReformulationQueriesC > nbReformulationQueriesA):
            nbReformulationQueriesA = nbReformulationQueriesC

        answer = "just try to find it" #input()
        if answer == "just try to find it": # train a model to detect similar expressions
          return "FindC", []
        else:
          res, c, scope_ids = getConstraint(answer)

    else: # TODO: manage the case where we couldn't find some parameters and the case of inccorrect prediction; more questions may be asked to the user
      # when we fire regular findC Also in partial queries (inside findC and findScope) use nlp !!
      print("I couldn't understand that, would you reformulate it ? or let me try to find it")
      # here a metric of number of constraints found by regular quack
      nbReformulationQueriesC += 1
      if(newsizenlpqueries3 > oldsizenlpqueries3):
        oldsizenlpqueries3 = newsizenlpqueries3
        if(nbReformulationQueriesC > nbReformulationQueriesA):
          nbReformulationQueriesA = nbReformulationQueriesC

      answer = "just try to find it" #input()
      if answer == "just try to find it": # train a model to detect similar expressions
        return "FindC", []
      else:
        res, c, scope_ids = getConstraint(answer)

## QuAcqBert

In [ ]:
def QuAcq2(B):
  global Target
  global problem_data
  global redundants
  global nbExamplesc
  global nbExamplesa
  global oldsizeexamples
  global newsizeexamples
  global Exampletimes
  global newsize
  global byNLP
  global newsizenlpqueries1
  global newsizenlpqueries2
  global newsizenlpqueries3
  global Bbk

  print("In QuAcq2: len of constraints in B is: ", len(B.constraints))
  L = Network(problem_data,[])
  cpt = 0
  while(True):

    stg = time.time()
    e = generateExample(list(problem_data.keys()),L, B)
    #register example times to generate them
    Exampletimes.append(time.time()-stg)

    #calculate number of examples generated until convergence and until finding an equivalent network
    nbExamplesc+=1
    if(newsizeexamples > oldsizeexamples):
      oldsizeexamples = newsizeexamples
      if(nbExamplesc > nbExamplesa):
        nbExamplesa = nbExamplesc


    if(e==[]):
      return True, L # convergence


    res, cons = ask(e, Target)

    if(res):
      # print(e," was classified positive positive")
      B = B.removeConstraintsThatRejectE(e)
      print("\nQuacq B reduced in size, new size: ",len(B.constraints))
    else:
      # print(e," was classified negative")
      res, scope_ids = conversation(e, cons)
      if(res=="FindC"):
        # think of calling RegularQuAcq2 here with original basis and L as background knowledge
        #FindC(e, FindScope(e , [] , list(problem_data.keys()), B ), L, B)
        scope_ids = RegularFindC(e, FindScope(e , [] , list(problem_data.keys()), Bbk ), L, Bbk)
        B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(scope_ids).constraints)
      elif(res=="Converged"):
        return True, L # convergence
      else:
        L = L.addConstraint(res)
        print("The nlp added constraint: \n")
        print(res.rel, res.scope_ids)
        print("!!!!!!!!!!!!!!!!!!!!!!!!!\n\n")
        byNLP+=1
        newsize+=1
        newsizeexamples+=1
        newsizenlpqueries1+=1
        newsizenlpqueries2+=1
        newsizenlpqueries3+=1
        B = B.removeListOfConstraints(B.ConstraintsIsExactlyY(scope_ids).constraints)
        print("L after nlp: ", len(L.constraints))

  return L


#ExperimentJigsaw

## One run

In [ ]:
# import pickle

# filepath = "/content/constraintsNLPgolomb4.txt"
# coordspath = "/content/golombCoods.txt"

# nlpConstraints = []
# with open(filepath, "r") as f:
#   for l in f.readlines():
#     nlpConstraints.append(l)

# coords = []
# with open(coordspath, "rb") as f:
#   coords = pickle.load(f)

# print(coords)

[{'scope': (1, 2), 'rel': <built-in function ne>, 'indice': 0}, {'scope': (1, 3), 'rel': <built-in function ne>, 'indice': 1}, {'scope': (1, 4), 'rel': <built-in function ne>, 'indice': 2}, {'scope': (2, 3), 'rel': <built-in function ne>, 'indice': 3}, {'scope': (2, 4), 'rel': <built-in function ne>, 'indice': 4}, {'scope': (3, 4), 'rel': <built-in function ne>, 'indice': 5}, {'scope': (5, 6), 'rel': <built-in function ne>, 'indice': 6}, {'scope': (5, 7), 'rel': <built-in function ne>, 'indice': 7}, {'scope': (5, 8), 'rel': <built-in function ne>, 'indice': 8}, {'scope': (6, 7), 'rel': <built-in function ne>, 'indice': 9}, {'scope': (6, 8), 'rel': <built-in function ne>, 'indice': 10}, {'scope': (7, 8), 'rel': <built-in function ne>, 'indice': 11}, {'scope': (9, 10), 'rel': <built-in function ne>, 'indice': 12}, {'scope': (9, 11), 'rel': <built-in function ne>, 'indice': 13}, {'scope': (9, 12), 'rel': <built-in function ne>, 'indice': 14}, {'scope': (10, 11), 'rel': <built-in function 

In [ ]:
# calls= 0

# from random import random
# from math import *

# n = 9
# vars_ids =   [i+1 for i in range(n**2)]
# vars_names = ["cell"+ str(i+1) for i in range(n**2)]
# vars_types = ["cell" for i in range(n**2)]
# vars_domains = [(1,n) for i in range(n**2)]
# types = ["cell"]

# problem_data = {
#   id: {
#       "domain": vars_domains[id-1],
#       "type": vars_types[id-1],
#       "name": vars_names[id-1]
#   } for id in vars_ids
# }



# language = [
#     (operator.ge, 2, False,None),
#     (operator.le, 2, False,None),
#     (operator.gt, 2, False,None),
#     (operator.lt, 2, False,None),
#     (operator.ne, 2, True,None),
#     (operator.eq, 2, True,None),
# ]

# constraints = []

# # lines :
# m = []

# for j in range(n):
#   l = []
#   for i in range(j*(int(sqrt(n))**2)+1, ((j+1)*(int(sqrt(n))**2))+1):
#     l.append(i)

#   m.append(np.array(l))
#   for p in combinations(l,2):
#     #print("p: ",p)
#     constraints.append(Constraint(list(p), operator.ne, 2, True))


# #colomns
# #print("\n________________\n")
# mm = np.array(m).T

# for i in range(n):
#   l = []
#   for j in range(n):
#     l.append(mm[i][j])

# #  print(l)
#   for p in combinations(l,2):
#     constraints.append(Constraint([int(p[0]), int(p[1])], operator.ne, 2, True))


# # blocks  = []
# # for k in range(0, n-int(sqrt(n))+1, int(sqrt(n))):
# #   for l in range(0, n-int(sqrt(n))+1, int(sqrt(n))):
# #     b = []
# #     for i in range(k, k+int(sqrt(n))):
# #       ll = []
# #       for j in range(l, l+int(sqrt(n))):
# #         #print(m[i][j])
# #         ll.append(m[i][j])
# #       b.append((ll))

# #     bb = []
# #     for i in range(len(b)):
# #       for j in range(len(b)):
# #         bb.append(b[i][j])

# #     for p in combinations(bb,2):
# #         constraints.append(Constraint([int(p[0]),int(p[1])], operator.ne, 2, True))

# #     blocks.append(b)

# Shapes = [
#   [1,2,3,10,11,12,19,20,21],
#   [8,9,17,18,27,36,45,54,53],
#   [24,25,26,33,34,35,42,43,44],
#   [28,29,37,46,55,64,65,73,74],
#   [38,39,40,47,48,49,56,57,58],
#   [59,66,67,68,69,75,76,77,78],
#   [22,30,31,32,41,50,51,52,60],
#   [4,5,6,7,13,14,15,16,23],
#   [61,62,63,70,71,72,79,80,81]
# ]

# for sh in Shapes:
#   for p in combinations(sh,2):
#     constraints.append(Constraint([int(p[0]),int(p[1])], operator.ne, 2, True))



# Target = Network(problem_data, constraints)
# Target = Target.removeDuplicates()
# ltarget = len(Target.constraints)

# B = Basis(problem_data,language)
# B.build()
# B = B.removeDuplicates()
# lbasis = len(B.constraints)


In [ ]:
# Qc = 0
# Qa = 0

# nbExamplesa = 0
# nbExamplesc = 0


# Exampletimes = []
# Examplesizes = []

# ClassifyTimes = []
# NERTimes = []


# nbClassifQueries = 0
# nbWhatIsWrongQueries = 0
# nbConfirmationQueries = 0

# oldsize = 0
# newsize = 0

# oldsizeexamples = 0
# newsizeexamples = 0

# waittimes = []


# ttts = time.time()
# maxwaittimeold = time.time()


# byNLP = 0
# byQuAcq = 0

# res = QuAcq2(B)
# ttte = time.time() - ttts

# averageGenerationTime = sum(Exampletimes)/len(Exampletimes)
# averageExampleSize = sum(Examplesizes) / len(Examplesizes)

# cumulativeWaitingUntilLearning = sum(Exampletimes[:nbExamplesa])
# totalCumulativeWaiting = sum(Exampletimes[:nbExamplesc])

In [ ]:
# p={
#         "Asolution":res[1].solve(),
#         "isAccepted":Target.isAccepted(res[1].solve())[0],
#         "TargetEquivalentToLearned": Target.isEquivalentTo(res[1]),
#         "sizeOfL":len(res[1].constraints),
#         "sizeOfBasis":lbasis,
#         "sizeOfTarget":ltarget,
#         "QA":Qa,
#         "QC":Qc,
#         "nbExamplesa":nbExamplesa,
#         "nbExamplesc":nbExamplesc,
#         "tmax":max(waittimes),
#         "t":averageGenerationTime,
#         "averageExampleSize":averageExampleSize,
#         "timeA":cumulativeWaitingUntilLearning,
#         "timeC":totalCumulativeWaiting,
#         "runningTime":ttte,
#         "byQuAcq": byQuAcq,
#         "byNLP": byNLP,
#         "TotalClassifyTimes" : sum(ClassifyTimes),
#         "AverageClassifyTimes" : sum(ClassifyTimes)/len(ClassifyTimes),
#         "TotalNERTimes" : sum(NERTimes),
#         "AverageNERTimes" : sum(NERTimes)/len(NERTimes),
#         "nbClassifQueries" : nbClassifQueries,
#         "nbWhatIsWrongQueries":nbWhatIsWrongQueries,
#         "nbConfirmationQueries": nbConfirmationQueries
#     }
# p

## 10 runs

In [ ]:
import pickle

filepath = "/content/constraintsNLPjigsaw!N!.txt"
coordspath = "/content/jigsawCoods.txt"

nlpConstraints10files = []
for i in range(10):
  nlpConstraints10files.append([])
  tmp = re.sub("!N!",str(i+1),filepath)
  with open(tmp, "r") as f:
    for l in f.readlines():
      nlpConstraints10files[i].append(l)

coordsALL = []
with open(coordspath, "rb") as f:
  coordsALL = pickle.load(f)

print(coordsALL)

[{'scope': (1, 2), 'rel': <built-in function ne>, 'indice': 0}, {'scope': (1, 3), 'rel': <built-in function ne>, 'indice': 1}, {'scope': (1, 4), 'rel': <built-in function ne>, 'indice': 2}, {'scope': (2, 3), 'rel': <built-in function ne>, 'indice': 3}, {'scope': (2, 4), 'rel': <built-in function ne>, 'indice': 4}, {'scope': (3, 4), 'rel': <built-in function ne>, 'indice': 5}, {'scope': (5, 6), 'rel': <built-in function ne>, 'indice': 6}, {'scope': (5, 7), 'rel': <built-in function ne>, 'indice': 7}, {'scope': (5, 8), 'rel': <built-in function ne>, 'indice': 8}, {'scope': (6, 7), 'rel': <built-in function ne>, 'indice': 9}, {'scope': (6, 8), 'rel': <built-in function ne>, 'indice': 10}, {'scope': (7, 8), 'rel': <built-in function ne>, 'indice': 11}, {'scope': (9, 10), 'rel': <built-in function ne>, 'indice': 12}, {'scope': (9, 11), 'rel': <built-in function ne>, 'indice': 13}, {'scope': (9, 12), 'rel': <built-in function ne>, 'indice': 14}, {'scope': (10, 11), 'rel': <built-in function 

In [ ]:
calls= 0

from random import random
from math import *


ressss = []


n = 9
vars_ids =   [i+1 for i in range(n**2)]
vars_names = ["cell"+ str(i+1) for i in range(n**2)]
vars_types = ["cell" for i in range(n**2)]
vars_domains = [(1,n) for i in range(n**2)]
types = ["cell"]

problem_data = {
  id: {
      "domain": vars_domains[id-1],
      "type": vars_types[id-1],
      "name": vars_names[id-1]
  } for id in vars_ids
}



language = [
    (operator.ge, 2, False,None),
    (operator.le, 2, False,None),
    (operator.gt, 2, False,None),
    (operator.lt, 2, False,None),
    (operator.ne, 2, True,None),
    (operator.eq, 2, True,None),
]

constraints = []

# lines :
m = []

for j in range(n):
  l = []
  for i in range(j*(int(sqrt(n))**2)+1, ((j+1)*(int(sqrt(n))**2))+1):
    l.append(i)

  m.append(np.array(l))
  for p in combinations(l,2):
    #print("p: ",p)
    constraints.append(Constraint(list(p), operator.ne, 2, True))


#colomns
#print("\n________________\n")
mm = np.array(m).T

for i in range(n):
  l = []
  for j in range(n):
    l.append(mm[i][j])

#  print(l)
  for p in combinations(l,2):
    constraints.append(Constraint([int(p[0]), int(p[1])], operator.ne, 2, True))


# blocks  = []
# for k in range(0, n-int(sqrt(n))+1, int(sqrt(n))):
#   for l in range(0, n-int(sqrt(n))+1, int(sqrt(n))):
#     b = []
#     for i in range(k, k+int(sqrt(n))):
#       ll = []
#       for j in range(l, l+int(sqrt(n))):
#         #print(m[i][j])
#         ll.append(m[i][j])
#       b.append((ll))

#     bb = []
#     for i in range(len(b)):
#       for j in range(len(b)):
#         bb.append(b[i][j])

#     for p in combinations(bb,2):
#         constraints.append(Constraint([int(p[0]),int(p[1])], operator.ne, 2, True))

#     blocks.append(b)

Shapes = [
  [1,2,3,10,11,12,19,20,21],
  [8,9,17,18,27,36,45,54,53],
  [24,25,26,33,34,35,42,43,44],
  [28,29,37,46,55,64,65,73,74],
  [38,39,40,47,48,49,56,57,58],
  [59,66,67,68,69,75,76,77,78],
  [22,30,31,32,41,50,51,52,60],
  [4,5,6,7,13,14,15,16,23],
  [61,62,63,70,71,72,79,80,81]
]

for sh in Shapes:
  for p in combinations(sh,2):
    constraints.append(Constraint([int(p[0]),int(p[1])], operator.ne, 2, True))



Targetorigin = Network(problem_data, constraints)
Targetorigin = Targetorigin.removeDuplicates()
ltarget = len(Targetorigin.constraints)

Borigin = Basis(problem_data,language)
Borigin.build()
Borigin = Borigin.removeDuplicates()
lbasis = len(Borigin.constraints)



for i in range(10):


  nlpConstraints = nlpConstraints10files[i]
  coords = deepcopy(coordsALL)

  B = deepcopy(Borigin)
  Bbk = deepcopy(B)

  Target = deepcopy(Targetorigin)


  Qc = 0
  Qa = 0

  nbExamplesa = 0
  nbExamplesc = 0


  Exampletimes = []
  Examplesizes = []

  ClassifyTimes = []
  NERTimes = []


  nbWhatIsWrongQueriesA = 0
  nbWhatIsWrongQueriesC = 0

  nbConfirmationQueriesA = 0
  nbConfirmationQueriesC = 0

  nbReformulationQueriesA = 0
  nbReformulationQueriesC = 0

  oldsize = 0
  newsize = 0

  oldsizeexamples = 0
  newsizeexamples = 0

  oldsizenlpqueries1 = 0
  newsizenlpqueries1 = 0

  oldsizenlpqueries2 = 0
  newsizenlpqueries2 = 0

  oldsizenlpqueries3 = 0
  newsizenlpqueries3 = 0

  waittimes = []


  ttts = time.time()
  maxwaittimeold = time.time()


  byNLP = 0
  byQuAcq = 0

  res = QuAcq2(B)
  ttte = time.time() - ttts

  averageGenerationTime = sum(Exampletimes)/len(Exampletimes)
  averageExampleSize = sum(Examplesizes) / len(Examplesizes)

  cumulativeWaitingUntilLearning = sum(Exampletimes[:nbExamplesa])
  totalCumulativeWaiting = sum(Exampletimes[:nbExamplesc])


  ressss.append({
        "Asolution":res[1].solve(),
        "isAccepted":Target.isAccepted(res[1].solve())[0],
        "TargetEquivalentToLearned": Target.isEquivalentTo(res[1]),
        "Learned": res[1],
        "sizeOfL":len(res[1].constraints),
        "sizeOfBasis":lbasis,
        "sizeOfTarget":ltarget,
        "QA":Qa,
        "QC":Qc,
        "nbWhatIsWrongQueriesA":nbWhatIsWrongQueriesA,
        "nbWhatIsWrongQueriesC":nbWhatIsWrongQueriesC,
        "nbConfirmationQueriesA":nbConfirmationQueriesA,
        "nbConfirmationQueriesC":nbConfirmationQueriesC,
        "nbReformulationQueriesA":nbReformulationQueriesA,
        "nbReformulationQueriesC":nbReformulationQueriesC,
        "nbExamplesa":nbExamplesa,
        "nbExamplesc":nbExamplesc,
        "tmax":max(waittimes),
        "t":averageGenerationTime,
        "averageExampleSize":averageExampleSize,
        "timeA":cumulativeWaitingUntilLearning,
        "timeC":totalCumulativeWaiting,
        "runningTime":ttte,
        "byQuAcq": byQuAcq,
        "byNLP": byNLP,
        "TotalClassifyTimes" : sum(ClassifyTimes),
        "AverageClassifyTimes" : sum(ClassifyTimes)/len(ClassifyTimes),
        "TotalNERTimes" : sum(NERTimes),
        "AverageNERTimes" : sum(NERTimes)/len(NERTimes)
    })

  print("\n\n\n Another entry was added to ressss, filenb== ",str(i+1),"\n\n\n")

In QuAcq2: len of constraints in B is:  396

 a COMPLETE query was sent, the answer was NO


 Can you tell what is wrong with this example: 
[(1, 'family1', 2), (2, 'family2', 1), (3, 'family3', 1), (4, 'family4', 1), (5, 'bought1', 1), (6, 'bought2', 1), (7, 'bought3', 1), (8, 'bought4', 1), (9, 'paid1', 1), (10, 'paid2', 1), (11, 'paid3', 1), (12, 'paid4', 1)]



Here !!!!
0 [1, 2] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
1 [1, 3] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
2 [1, 4] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
3 [2, 3] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
4 [2, 4] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
5 [3, 4] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
6 [5, 6] <built-

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


I couldn't understand that, would you reformulate it ? or let me try to find it

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  393

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  390

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  61 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[2, 3], [2, 3]]
[<built-in function le>, <built-in function lt>] [[3, 2], [3, 2]]
<built-in function ne> [2, 3]


L[Y] before findEPrime: 


 Example:  [(2, 'family2', 2), (3, 'family3', 1)]


Delta returned from findEPrime: 

[<built-in function le>, <built-in function lt>] [[2, 3], [2, 3]]
[<built-in function le>, <built-in function lt>] [[3, 2], [3, 2]]
<built-in function ne> [2, 3]


L[Y] returned from findEPrime: 


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


[3, 5] <built-in function eq> 2 True
Is this what you meant ?:
 <built-in function eq> [3, 5] True None 2
The nlp added constraint: 

<built-in function eq> [3, 5]
!!!!!!!!!!!!!!!!!!!!!!!!!


L after nlp:  2

 a COMPLETE query was sent, the answer was NO


 Can you tell what is wrong with this example: 
[(1, 'family1', 2), (2, 'family2', 2), (3, 'family3', 1), (4, 'family4', 1), (5, 'bought1', 1), (6, 'bought2', 1), (7, 'bought3', 1), (8, 'bought4', 1), (9, 'paid1', 1), (10, 'paid2', 1), (11, 'paid3', 1), (12, 'paid4', 1)]



Here !!!!
0 [1, 2] <built-in function ne> 

[2, 7] <built-in function eq> None 

----------

False
Here !!!!
1 [1, 3] <built-in function ne> 

[2, 7] <built-in function eq> None 

----------

False
Here !!!!
2 [1, 4] <built-in function ne> 

[2, 7] <built-in function eq> None 

----------

False
Here !!!!
3 [2, 3] <built-in function ne> 

[2, 7] <built-in function eq> None 

----------

False
Here !!!!
4 [2, 4] <built-in function ne> 

[2, 7] <built-in function eq

...

[12, 10] <built-in function lt> 2 False
Is this what you meant ?:
 <built-in function lt> [12, 10] False None 2

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  271

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  339 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[6, 5], [6, 5]]
<built-in function ne> [5, 6]


L[Y] before findEPrime: 


 Example:  [(5, 'bought1', 2), (6, 'bought2', 4)]


Delta returned from findEPrime: 

[<built-in function le>, <built-in function lt>] [[6, 5], [6, 5]]
<built-in function ne> [5, 6]


L[Y] returned from findEPrime: 


ENDENDENDENDENDENDENDENDENDENDEND findEPrimeCall:  339 ---------------------




 a PARTIAL query was sent, the answer was YES

 FindC : B reduced in size, new size:  269

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  340 ---------------------





Delta before findEPrime: 

<buil

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


[12, 10] <built-in function lt> 2 False
Is this what you meant ?:
 <built-in function lt> [12, 10] False None 2

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  264

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was NO

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  252

 a PARTIAL query was sent, the answer was YES

Inside FindScope B reduced in size, new size:  246

STARTSTARTSTARTSTARTSTARTSTARTSTART findEPrimeCall:  341 ---------------------





Delta before findEPrime: 

[<built-in function le>, <built-in function lt>] [[7, 8], [7, 8]]
[<built-in function le>, <built-in function lt>] [[8, 7], [8, 7]]
<built-in function ne> [7, 8]


L[Y] before findEPrime: 


 Example:  [(8, 'bought4', 1), (7, 'bought3', 2)]


Delta returned from findEPrime: 

[<built-in function le>, <built-in function lt>] [[7, 8], [7, 8]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2377: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


[8, 12] <built-in function eq> 2 True
Is this what you meant ?:
 <built-in function eq> [8, 12] True None 2
The nlp added constraint: 

<built-in function eq> [8, 12]
!!!!!!!!!!!!!!!!!!!!!!!!!


L after nlp:  16

 a COMPLETE query was sent, the answer was NO


 Can you tell what is wrong with this example: 
[(1, 'family1', 4), (2, 'family2', 1), (3, 'family3', 2), (4, 'family4', 3), (5, 'bought1', 2), (6, 'bought2', 4), (7, 'bought3', 1), (8, 'bought4', 2), (9, 'paid1', 1), (10, 'paid2', 1), (11, 'paid3', 2), (12, 'paid4', 2)]



Here !!!!
0 [1, 2] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
1 [1, 3] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
2 [1, 4] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
3 [2, 3] <built-in function ne> 

[12, 10] <built-in function lt> None 

----------

False
Here !!!!
4 [2, 4] <built-in function ne> 

[12, 10] <built-

In [ ]:
import pickle

with open("resjigsaw10runs10files.txt", "wb") as f:
  pickle.dump(ressss,f)
